### Modeling optical selection bias on cluster density profile

Author: M. Costanzi

Date: 01/01/2026

Notebook to compute:

- optical cluster bias
- optical cluster density profile

The notebook uses $\texttt{CAMB}$ to compute power spectra

### Density profile:

\begin{equation}
\langle \Sigma (R|\lambda^{\rm ob},z^{\rm ob}) \rangle = \langle \Sigma^{\rm 1-halo} (R|\lambda^{\rm ob},z^{\rm ob}) \rangle + \langle \Sigma^{\rm prj} (R|\lambda^{\rm ob},z^{\rm ob}) \rangle
\end{equation}

\begin{multline}
    \langle \Sigma^{\rm prj} (R|\lambda^{\rm ob},z^{\rm ob}) \rangle = \int_0^{\infty} d z\ \int_0^{\theta_{\rm mis, max}} d \theta_{\rm mis}  \frac{d V}{d z d \Omega} 
 2 \pi \sin(\theta_{\rm mis})  \int_0^{\infty} d M\ n(M,z) \times \\ [1 + b(M,z) \langle b (\lambda^{\rm ob}, z^{\rm ob},\theta_{\rm mis}) \rangle \xi_{\rm NL}(|r(z)-r(z^{\rm ob})|,\bar{z})] \Sigma_{\rm mis} (R |M,z, R(\theta_{\rm mis})) \, ,
\end{multline}

where: 
\begin{equation}
\Sigma_{\rm mis} (R |M,  R(\theta_{\rm mis})) = \frac 1 {2 \pi} \int_0^{2 \pi} \Sigma \, \bigg( \sqrt {R^2 +  R(\theta_{\rm mis})^2 - 2 R  R(\theta_{\rm mis}) \cos \varphi} | M,z \bigg) \, {\rm d} \varphi,
\end{equation}

\begin{equation}
    |{r}(z)-{r}(z^{\rm ob})| = \sqrt{\chi(z)^2+\chi(z^{\rm ob})^2 - 2\chi(z)\chi(z^{\rm ob})\cos (\theta_{\rm mis}) } \, .
\end{equation}

\begin{equation}
 \langle b (\lambda^{\rm ob}, z^{\rm ob},\theta_{\rm mis}) \rangle \propto \int d\lambda^{\rm tr} b_{\rm sel}(\lambda^{\rm ob},\lambda^{\rm tr},z,\theta_{\rm mis}) P(\lambda^{\rm ob}|\lambda^{\rm tr},z^{\rm ob})n(\lambda^{\rm tr},z^{\rm ob})
\end{equation}

### Otical cluster bias:

To compute $b_{\rm sel}(\lambda^{\rm ob},\lambda^{\rm tr},z)$, consider the mean boost on richness due to projected structures for a cluster having $\lambda^{\rm ob},\lambda^{\rm tr}$:

\begin{multline}
\Delta^{\rm prj}(\lambda^{\rm ob},\lambda^{\rm tr},z^{\rm ob}) = \lambda^{\rm ob}-\lambda^{\rm tr} =\int dz \frac{d V}{d z d \Omega} w(z-z^{\rm ob},\sigma_z(z)) \int_0^{\lambda^{\rm ob}} d\lambda f(\lambda^{\rm ob},z^{\rm ob},\lambda,z) \lambda \cdot \\ 2 \pi \int_0^{\theta(\lambda^{\rm ob})+\theta(\lambda)} d \theta \sin(\theta) \int dM n(M,z)P(\lambda|M,z)  [1 + b(M,z) b_{\rm sel}(\lambda^{\rm ob},\lambda^{\rm tr},z) \xi_{\rm NL}(|r(z)-r(z^{\rm ob})|,\bar{z})]
\end{multline}

Thus, expliciting $b_{\rm sel}(\lambda^{\rm ob},\lambda^{\rm tr},z^{\rm ob}) $:

\begin{equation}
 b_{\rm sel}(\lambda^{\rm ob},\lambda^{\rm tr},z^{\rm ob})= \\= \frac{ \Delta^{\rm prj}(\lambda^{\rm ob},\lambda^{\rm tr},z)-\bar{\Delta}^{\rm prj}_{\rm bkg}(\lambda^{\rm ob},z)}{\int dz \frac{d V}{d z d \Omega} w(z-z^{\rm ob},\sigma_z(z)) \int_0^{\lambda^{\rm ob}} d\lambda f(\lambda^{\rm ob},z^{\rm ob},\lambda,z) \lambda 2 \pi \int_0^{\theta(\lambda^{\rm ob})+\theta(\lambda)} d \theta \sin(\theta) \int dM n(M,z)P(\lambda|M,z) b(M,z)\xi_{\rm NL}(|r(z)-r(z^{\rm ob})|,\bar{z})}
\end{equation}

\begin{equation}
\Delta^{\rm prj}(\lambda^{\rm ob},\lambda^{\rm tr},z^{\rm ob}) = \lambda^{\rm ob}-\lambda^{\rm tr}
\end{equation}

\begin{equation}
\bar{\Delta}^{\rm prj}_{\rm bkg}(\lambda^{\rm ob},z^{\rm ob}) = \int dz \frac{d V}{d z d \Omega} \int d\lambda \Omega(\lambda^{\rm ob},z^{\rm ob},\lambda,z) w(z-z^{\rm ob},\sigma_z(z))f(\lambda^{\rm ob},z^{\rm ob},\lambda,z) \lambda \int dM n(M,z)P(\lambda|M)
\end{equation}

In [ ]:
%matplotlib inline

from astropy.io import fits # per leggere fits file
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit,minimize
from scipy.integrate import quad
from scipy.interpolate import InterpolatedUnivariateSpline as ius # interpolation
from scipy.interpolate import interp1d
import scipy.stats as stats
import scipy.special as spc

In [ ]:
## see https://matplotlib.org/users/customizing.html to customize
import matplotlib
# matplotlib.rc('text', usetex=False)
matplotlib.rc('font', family='serif')
matplotlib.rc('figure',figsize=( 9.6*0.8, 6.8*0.8))
matplotlib.rc('legend',fontsize='x-large')
matplotlib.rc('xtick',direction='in')
matplotlib.rc('xtick',labelsize=16)
matplotlib.rc('xtick.major',size=7)
matplotlib.rc('xtick.minor',size=4)
matplotlib.rc('xtick.major',width=1.25)
matplotlib.rc('xtick.minor',width=1.25)
matplotlib.rc('ytick',direction='in')
matplotlib.rc('ytick',labelsize=16)
matplotlib.rc('ytick.major',size=7)
matplotlib.rc('ytick.minor',size=4)
matplotlib.rc('ytick.major',width=1.25)
matplotlib.rc('ytick.minor',width=1.25)
matplotlib.rc('patch',edgecolor='k' )
# matplotlib.rc('patch',force_edgecolor=True)
# matplotlib.rc('scatter',markeredgecolor='k')
matplotlib.rc('legend',markerscale=1.5)
matplotlib.rc('errorbar',capsize = 3)
matplotlib.rc('axes',labelsize='x-large')
matplotlib.rc('axes',linewidth=1.25)

In [ ]:
# Define Cosmology


# # # =============|
# # # Buzzard v1.1 |
# # # =============|
Omega_m=0.286
h=0.7
Omega_b=0.046
n_s=0.96
sigma_8=0.82


# Constants
c_light=2997.92458 #[km/s/100]
rho_c=2.7751428946e11 # =3/8*100 2 /pi/G*Mpc/M_sun * 1e6 [in units M_sun / Mpc^3 at z=0]
delta_cr = 1.686

In [ ]:
## ===============================================================================
## COMPUTE ANGULAR SIZE OF HALOS =================================================
## Theta = Physical_size / D_a(z) ================================================
## D_a(z) = D_com_transverse(z)/(1+z) ============================================
## D_com_transverse(z) = D_com(z) FOR OMEGA_k=0 ==================================
## D_com(z)= c/H0 \int_0^z dz' / E(z') =========================================== 
	# :::::: E(z) :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    #E(z)^2=O_m * (1+z)^3 + (1- O_m - O_l) * (1+z)^2 + O_l * (1+z)^3 * exp [-3 int_1^a eosDE / a da]=
    #      = O_m / a^3 + (1- O_m - O_l)/a^2 + O_l/a^3 * exp [-3 int_1^a eosDE / a da]
def e(z,omem): # input: scale factor, omega matter
    a=1.0/(1.0+z)
    return np.sqrt(omem/a**3.+(1.-omem)) #for Omega_m + Omega_l = 1.0 & w_0=-1 & w_a=0
	#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

	# :::::: 1/E(z) :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    #E(z)^2=O_m * (1+z)^3 + (1- O_m - O_l) * (1+z)^2 + O_l * (1+z)^3 * exp [-3 int_1^a eosDE / a da]=
    #      = O_m / a^3 + (1- O_m - O_l)/a^2 + O_l/a^3 * exp [-3 int_1^a eosDE / a da]
def f_inv_e(z,omem): # input: redshift, omega matter
    a=1.0/(1.0+z)
    return 1./np.sqrt(omem/a**3.+(1.-omem)) #for Omega_m + Omega_l = 1.0 & w_0=-1 & w_a=0

# :::::: E(z)^2 :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
#     E(z)^2=O_m * (1+z)^3 + (1- O_m - O_l) * (1+z)^2 + O_l * (1+z)^3 * exp [-3 int_1^a eosDE / a da]=
#           = O_m / a^3 + (1- O_m - O_l)/a^2 + O_l/a^3 * exp [-3 int_1^a eosDE / a da]
def e_a_sq(a): # input: scale factor
    espo=-3. *(1. +w+wa) # per w_0 /= 1 & w_a /= 0
    e=omega_m/(a**3. )+(omega_k)/(a**2. )+omega_v*a**espo*np.exp(3. *wa*(a-1. )) # for w_0 /= 1 & w_a /= 0
    return e
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

	#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
	#====== COMOVING DISTANCE @ Z =================================================
    # D_com(z)= c/H0 \int_0^z dz' / E(z') =========================================
def d_com(z,omem): # input redshift, omega matter
    integrale=quad(f_inv_e, 0.0,z,args=(omem))
    return (2997.92458)*integrale[0]

def d_a(z,omem):
    return d_com(z,omem)/(1.+z)

def theta(size,z,omem): # angular size in radians
    thetas=size/d_a(z,omem)
    return thetas

	#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
	#====== COMOVING VOLUME ELEMENT PER UNIT SOLID ANGLE AND REDSHIFT =========================
    # d V / d Omega dz= (c/H_0)^3 * 1/|Omega_k| [sinn(sqrt(|Omega_k|) * int_0^z 1/E(z) dz)]^2 /E(z)
def derivs_com_dis(z,omem): # input redshift, omega matter
#    return (2997.92458)**3.*(omem*(1.+z)**3.+1.-omem)**(-0.5) # for w_0=-1 & w_a=0 & O_k=0
    integrale=quad(f_inv_e, 0.0,z,args=(omem))
    return (2997.92458)**3.0*f_inv_e(z,omem)*integrale[0]**2.0


# from chi to z convert function
z_chi_grid=np.linspace(0.0,2.0,200)
d_com_4interp=[d_com(z_chi_grid[i],Omega_m) for i in range(len(z_chi_grid))]
d_com_4interp=np.array(d_com_4interp)
z_of_chi=ius(d_com_4interp,z_chi_grid,k=2)
chi_of_z=ius(z_chi_grid,d_com_4interp,k=2) # [cMpc/h]

derivs_com_dis_4interp=np.array([derivs_com_dis(z_chi_grid[i],Omega_m) for i in range(z_chi_grid.size)])
dVdzdOmg = ius(z_chi_grid,derivs_com_dis_4interp,k=2) # [cMpc^3/h^3]

In [ ]:
import camb
from camb import model, initialpower
print('CAMB version: %s '%camb.__version__)

In [ ]:


# Initialize Tinker interpolation (A, a, b, c)
x = np.log((200., 300., 400., 600., 800., 1200., 1600., 2400., 3200.))
y = (1.858659e-01, 1.995973e-01, 2.115659e-01, 2.184113e-01, 2.480968e-01, 2.546053e-01, 2.600000e-01, 2.600000e-01, 2.600000e-01)
interp_A = interp1d(x, y, kind='cubic')
y = (1.466904e+00, 1.521782e+00, 1.559186e+00, 1.614585e+00, 1.869936e+00, 2.128056e+00, 2.301275e+00, 2.529241e+00, 2.661983e+00)
interp_a = interp1d(x, y, kind='cubic')
y = (2.571104e+00, 2.254217e+00, 2.048674e+00, 1.869559e+00, 1.588649e+00, 1.507134e+00, 1.464374e+00, 1.436827e+00, 1.405210e+00)
interp_b = interp1d(x, y, kind='cubic')
y = (1.193958e+00, 1.270316e+00, 1.335191e+00, 1.446266e+00, 1.581345e+00, 1.795050e+00, 1.965613e+00, 2.237466e+00, 2.439729e+00)
interp_c = interp1d(x, y, kind='cubic')


def Tinker_params(z, Deltamean):
    """For given redshift and mean overdensity, return list of four Tinker
    parameters. If outside defined overdensity, return last valid number."""
    # Parameters at z=0
    if Deltamean<=200:
        A, a, b, c = [.186, 1.47, 2.57, 1.19]
    elif Deltamean>=3200:
        A, a, b, c = [.26, 2.66, 1.41, 2.44]
    else:
        lnDeltamean = np.log(Deltamean)
        A, a, b, c = [interp_A(lnDeltamean), interp_a(lnDeltamean), interp_b(lnDeltamean), interp_c(lnDeltamean)]
    if Deltamean>=1600: A=0.26 # mod_costanzi, to avoid interpolation problem
    # Redshift evolution
    logalpha = -(.75/np.log10(Deltamean/75.))**1.2
    alpha = 10.**logalpha
    A*= (1.+z)**-.14
    a*= (1.+z)**-.06
    b*= (1.+z)**-alpha
    return A, a, b, c


# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# Halo Mass Function dn/dM= f(sigma(M,z))/(4/3pi R^3) * d Ln(sigma(M,z))^-1/dM
def dndM(halo_mass,z_redshift):
    Deltamean=200.
    Radius=M_to_R(halo_mass)
    sigma2_Rz=np.exp((Lnsigma2R_at_z(z_redshift,np.log(Radius)).T)[0]) # from interpolation
    dndM = fnu_nu(sigma2_Rz,z_redshift,Deltamean)/(4.*np.pi*Radius**3. /3. )*dlog_sigdm(Radius,z_redshift,sigma2_Rz)/halo_mass
    return dndM

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# Tinker et al. 2008 mass function
def fnu_nu(sigma2,z,Deltamean): #x=sigma^2 y=redshift
    A,a,b,c=Tinker_params(z, Deltamean)
    fnu_nu=A* ((sigma2**.5/b)**-a + 1.) * np.exp(-c/sigma2)
    return fnu_nu

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# d ln(sig^-1)/ d M = - d sig^2/ d R * 1/ sig^2 * R / (6*M)
def dlog_sigdm(x,z_redshift,sigma2_r_z): #x=rsigma (R relativo a sigma(R)),sigma2_r_z=sigma^2(R,z)
    dlog_sigdm=np.exp((Lndsigma2R_at_z(z_redshift,np.log(x)).T)[0])*x/sigma2_r_z/6. 
    return dlog_sigdm

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# d sig^2(R) / d R =1/(2*Pi*Pi) \int (dk k^3 P(k) * 2 * W(kR) * dW/dr)
def dsig2dr(r,z_redshift): #x=rsigma (R relativo a sigma(R))

    Ln_k_min=np.log(0.0001 )
    Ln_k_max=np.log(200. )

    ss,err=quad(d_dsig2dr, Ln_k_min,Ln_k_max,epsrel=1.0e-3,args=(r,z_redshift))
    return ss

def d_dsig2dr(x,rsmooth,z_red): #integranda di d sigma^2 / d R; x=wave number
    f_k=np.exp(x)
    R_k=f_k*rsmooth
    if (R_k < 5e-4):
        dwdr = 0. 
    else:
        dwdr=3.  * (np.sin(R_k) * (R_k**2.  - 3. ) + 3.  * R_k * np.cos(R_k)) / (R_k**4. ) #!derivata top_hat rispetto R
    d_dsig2dr=(f_k**4. )*Pk_at_z.P(z_red,f_k)*filtTH(R_k)*dwdr/np.pi/np.pi
    return d_dsig2dr


# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# sigma^2(R,z) = 1/2pi^2 int_0^inf dk k^2 P(k,z) W(kR)^2 = 1/2pi^2 int_-inf^inf dLn(k) k^3 P(k,z) W(kR)^2
def sigma_r(r,z_redshift):

    # Extrema of the logarithmic integral
    Ln_k_min=np.log(0.0001 )
    Ln_k_max=np.log(10. )

    ss,err=quad(dsigma_r, Ln_k_min,Ln_k_max, epsrel=1.0e-3,args=(r,z_redshift))
    sigma_r=ss/2.  /np.pi /np.pi
    return sigma_r

def dsigma_r(x,rsmooth,z_red): # d sigma^2(R,z) = k^3 P(k,z) W(kR)^2 ; input: x= Ln(k)

    wave_num=np.exp(x) # wave_num = k
    rk=rsmooth*wave_num
    dsigma_r=wave_num**3.  * Pk_at_z.P(z_red,wave_num)*filtTH(rk)**2. 
    return dsigma_r

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def filtTH(rk): # Top-Hat filter k-space
    if (rk < 1.0e-6) :
        filtTH = 1.  #to avoid precision problems
    else :
        filtTH=3. *(np.sin(rk)-rk*np.cos(rk))/rk**3. 
    return filtTH

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

def M_to_R(halo_mass): # conversion from mass to radius: input units [M_sun/h]

    M_to_R = (halo_mass*3. /(4. *np.pi*rho_c*omega_dm))**(1./3.) # [Mpc/h] ; NB neutrino component neglected
    return M_to_R

#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def R_to_M(Radius): # conversion from radius to mass: input units [Mpc/h]

    R_to_M = 4. *np.pi*Radius**3. /3. *rho_c*omega_dm # [M_sun/h] ; NB neutrino component neglected
    return R_to_M
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
# Create interpolation table for sigma^2(r,z)
Log10MminHMF=12.0
Log10MmaxHMF=16.0
n_log_M_step=100 # number of points in mass where compute \sigma^2(R,z) and d\sigma^2(R,z)/dR

z_inf=0.05
z_sup=0.95

num_of_z_pk=19 # Number of z step used to compute the P_k
zs_pk=np.linspace(z_inf,z_sup,num_of_z_pk)
print(zs_pk)

# Define Cosmological params:
omega_m  = Omega_m
h0  =   h
ombh2 = Omega_b*h**2.
LnA_s = 3.054 # per sigma8=0.82 Buzzard v1.1
tau = 0.078
omega_k = 0.0
w = -1.0
wa = 0.0
omnuh2=0.0
massive_nu=0
massless_nu=3.046

#Consistency with CAMB input
H_0=h0*100.
omch2=omega_m*h0**2.-ombh2-omnuh2
A_s=np.exp(LnA_s)/1.0e10
mnu=94.57*omnuh2


# Usefull Consistentcy
omega_dm=omega_m-omnuh2/h0**2.
omega_v=1-omega_m-omega_k

pars = camb.CAMBparams()
pars.set_cosmology(H0=H_0, ombh2=ombh2, omch2=omch2,omk=omega_k,neutrino_hierarchy='degenerate', num_massive_neutrinos=3, mnu=mnu, nnu=3.046)
pars.set_dark_energy() #re-set defaults
pars.InitPower.set_params(As=A_s, ns=n_s)
pars.NonLinear = model.NonLinear_both
pars.DoLensing = False

Pk_at_z = camb.get_matter_power_interpolator(pars, zs=zs_pk, kmax=200, nonlinear=False,
                              var1=8, var2=8, hubble_units=True, k_hunit=True,
                              return_z_k=False, k_per_logint=None, log_interp=True)
#Non-Linear spectra (Halofit)
Pk_at_z_NonLin = camb.get_matter_power_interpolator(pars, zs=zs_pk, kmax=200, nonlinear=True,
                              var1=7, var2=7, hubble_units=True, k_hunit=True,
                              return_z_k=False, k_per_logint=None, log_interp=True)

#Now get matter power spectra and sigma8 at redshift 0
pars.set_matter_power(redshifts=[0.], kmax=100.0)
results = camb.get_results(pars)
s8 = np.array(results.get_sigma8())
print( s8)

# # Array for interpolation
log_M_min=np.log(10. **Log10MminHMF+1. )
log_M_max=np.log(10.**Log10MmaxHMF-1.) 


n_log_M_step=100 # number of points in mass where compute \sigma^2(R,z) and d\sigma^2(R,z)/dR
Ln_R_array=np.zeros(n_log_M_step)
Ln_M_array=np.zeros(n_log_M_step)
Ln_sigma2Rz=np.zeros((n_log_M_step,num_of_z_pk))
Ln_dsigma2Rz=np.zeros((n_log_M_step,num_of_z_pk))

d_logM=(log_M_max-log_M_min)/(n_log_M_step-1. ) 
# Compute and store \sigma^2(R,z) and d\sigma^2(R,z)/dR ===
for nm in range(n_log_M_step):
    log_mass=log_M_min+d_logM*(nm)
    Ln_M_array[nm]=log_mass
    Ln_R_array[nm]=np.log(M_to_R(np.exp(log_mass)))
    for nz in range(num_of_z_pk):
        Ln_sigma2Rz[nm,nz]=np.log(sigma_r(M_to_R(np.exp(log_mass)),zs_pk[nz]))
        Ln_dsigma2Rz[nm,nz]=np.log(-dsig2dr(M_to_R(np.exp(log_mass)),zs_pk[nz])) # NB disgma2/dR is negative

Lnsigma2R_at_z=interpolate.interp2d(zs_pk,Ln_R_array,Ln_sigma2Rz, kind='cubic')
Lndsigma2R_at_z=interpolate.interp2d(zs_pk,Ln_R_array,Ln_dsigma2Rz, kind='cubic')


In [ ]:
def bias_halo(M,z,model='tinker'):
    '''
    Halo bias
    if model=tinker M200,m [Msun/h]
    if model=castro Mvir,c [Msun/h]
    '''
    if model=='tinker':
        Radius=M_to_R(M)
        sigma2_Rz=np.exp((Lnsigma2R_at_z(z,np.log(Radius)).T)[0]) # sigma^2 from interpolation
        fy=delta_cr /np.sqrt(sigma2_Rz) # delta_cr/sigma(R,z)
        Delta_m= 200. # Delta w.r.t. the mean density
        y=np.log10(Delta_m) #log10(Delta)
        flarge_A = 1. +0.24 *y*np.exp(-(4. /y)**4. )
        fsmall_A = 0.44 *y-0.88 
        flarge_B = 0.183 
        fsmall_B = 1.5 
        flarge_C = 0.019 +0.107 *y+0.19 *np.exp(-(4. /y)**4. )
        fsmall_C = 2.4 
        bias_tin=1. -flarge_A*fy**fsmall_A/(fy**fsmall_A+delta_cr**fsmall_A)+flarge_B*fy**fsmall_B+flarge_C*fy**fsmall_C
        return bias_tin#*0.95    
    if model=='castro':
        a_1=0.7962
        a_2=0.1449
        a_z=-0.0658
        p_1=-0.5612
        p_2=-0.4743
        q_1=0.3688
        q_2=-0.2804
        q_z=0.0251
        a_scale=1./(1.+z)
        om_m_z=Omega_m*(1.+z)**3/e_a_sq(a_scale)
        delta_c_lin=3./20.*(12.*np.pi)**(2./3.)*(1.+0.012299*np.log(om_m_z))
        Radius=M_to_R(M)
        sigma2_Rz=np.exp((Lnsigma2R_at_z(z,np.log(Radius)).T)[0]) # sigma^2 from interpolation
        dlnsigdM=dlog_sigdm(Radius,z,sigma2_Rz)
        nu=delta_c_lin/np.sqrt(sigma2_Rz)    
        aR=a_1+a_2*(dlnsigdM+0.6125)**2.
        a=aR*om_m_z**a_z
        p=p_1+p_2*(dlnsigdM+0.5)
        qR=q_1+q_2*(dlnsigdM+0.5)
        q=qR*om_m_z**q_z
        bias= 1. + (a*nu**2.-q)/delta_c_lin + 2.*p/delta_c_lin/(1.+(a*nu**2.)**p)
        return bias        

In [ ]:
# Define intrinsic scaling relation lambda_true - M,z

# DESY1 NC+3x2pt BEST FIT
M_min=10.**1.13852818e+01
alpha=8.58693714e-01
M_1=10.**1.26964410e+01
M_pivot=M_1-M_min
sigm_intr=1.80949022e-01
epsi=2.83887020e-01

pivot_z0=0.4544

## Mock Observable-Lambda realtion
def l_sat(M,z,Mmin,Mpivot,a,epsilon):
    return ((M-Mmin)/Mpivot)**a*((1.+z)/(1.+pivot_z0))**epsilon

def l_tr(M,z):
    return 1.+l_sat(M,z,M_min,M_pivot,alpha,epsi)

def sig_intr(M,z):
    return sigm_intr*l_sat(M,z,M_min,M_pivot,alpha,epsi)


def pltr_M(ltr,M,z):
    m=l_sat(M,z,M_min,M_pivot,alpha,epsi)
    std=np.sqrt(m+(m*sigm_intr)**2.)
    x=ltr+(m*sigm_intr)**2.
    lam=std**2.
    ln_gamma_fun=spc.gammaln(x)
    return np.exp(-lam+(x-1.)*np.log(lam)-ln_gamma_fun,dtype='float128')

In [ ]:
# INTERPOLATION FUNCTION FOR \Xi_NonLinear(r [cMpc/h],z)

# Create interpolation table
r_s=10.**np.linspace(np.log10(0.0001),np.log10(250.),300) # cMpc/h
xi_r_NL_CAMB=np.zeros((zs_pk.size,r_s.size))

for iz in range(len(zs_pk)):
    for ir in range(len(r_s)):
        kh_grid=10.**np.linspace(-3.,2.,50000)
        dlnk_xi_r_CAMB=kh_grid**3.*Pk_at_z_NonLin.P(zs_pk[iz],kh_grid)*np.sin(kh_grid*r_s[ir],dtype='float64')/(kh_grid*r_s[ir])
        xi_r_NL_CAMB[iz,ir]=np.trapz(dlnk_xi_r_CAMB,np.log(kh_grid))/2./np.pi**2.

xir2_NL_r_z=interpolate.RectBivariateSpline(np.log(r_s),zs_pk,(xi_r_NL_CAMB*r_s**2.).T)


In [ ]:
# INTERPOLATION FUNCTION FOR \Xi_Linear(r [cMpc/h],z)

# Create interpolation table
r_s=10.**np.linspace(np.log10(0.0001),np.log10(250.),300) # cMpc/h
xi_r_CAMB=np.zeros((zs_pk.size,r_s.size))

for iz in range(len(zs_pk)):
    for ir in range(len(r_s)):
        kh_grid=10.**np.linspace(-3.,2.,50000)
        dlnk_xi_r_CAMB=kh_grid**3.*Pk_at_z.P(zs_pk[iz],kh_grid)*np.sin(kh_grid*r_s[ir],dtype='float64')/(kh_grid*r_s[ir])
        xi_r_CAMB[iz,ir]=np.trapz(dlnk_xi_r_CAMB,np.log(kh_grid))/2./np.pi**2.

        
xir2_r_z=interpolate.RectBivariateSpline(np.log(r_s),zs_pk,(xi_r_CAMB*r_s**2.).T)


In [ ]:
# FOR P(lob|ltr,z)=MC19 DES Y3 ===============================
z_bins=np.linspace(0.10,0.80,15)
params=np.loadtxt('prj_params_DESY3_lss_lin_dep_getdist_v1.txt').T
a_tau_fit=params[0,:]
b_tau_fit=params[1,:]
a_mu_fit=params[2,:]
b_mu_fit=params[3,:]
a_sig_fit=params[4,:]
b_sig_fit=params[5,:]
a_fprj_fit=params[6,:]
b_fprj_fit=params[7,:]
a_fmsk_fit=params[8,:]
b_fmsk_fit=params[9,:]
# Define interpolation functions to get a and b at a given redhsift
atau=ius(z_bins,a_tau_fit,k=1)
btau=ius(z_bins,b_tau_fit,k=1)
amu=ius(z_bins,a_mu_fit,k=1)
bmu=ius(z_bins,b_mu_fit,k=1)
asig=ius(z_bins,a_sig_fit,k=1)
bsig=ius(z_bins,b_sig_fit,k=1)
afprj=ius(z_bins,a_fprj_fit,k=1)
bfprj=ius(z_bins,b_fprj_fit,k=1)
afmsk=ius(z_bins,a_fmsk_fit,k=1)
bfmsk=ius(z_bins,b_fmsk_fit,k=1)


def mu_model(lin,a,b):
    return a+b*lin

def sig_model(lin,a,b):
    return b*lin**a

def tau_model(lin,a,b):
    return b/lin**a

#def fprj_model(lin,a,b):
#    return b*lin**a

def fprj_model(lin,a,b):
    return b/(1.+np.exp(-lin/1.))**a

def fmask_model(lin,a,b):
    return b*lin**a

In [ ]:
# DEFINE TERMS OF \rho_prj

def R_lambda(lam):
    return (lam/100.)**0.2

def f_area(lob,zob,ltr,z):
    theta_ltrz=R_lambda(ltr) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zob)/chi_of_z(zob) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    return (1.+theta_ltrz/theta_lobzob)**(-2.)

def Omega_halos(lob,zob,ltr,z):
    theta_ltrz=R_lambda(ltr) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zob)/chi_of_z(zob) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    return 2.*np.pi*(1.-np.cos(theta_ltrz+theta_lobzob))

def kernel_z(z,zob):
    l_mod=1. - (z-zob)**2./z_kernel_size(z)**2.
    l_mod[l_mod<0.]=0.
    return l_mod    

# Funzioni per trovare l'intervallo di redshift da considerare per proiezini
from scipy.optimize import bisect

def zmax4zkernel(zmax,z0):
    return zmax-z_kernel_size(zmax)-z0

def zmin4zkernel(zmin,z0):
    return zmin+z_kernel_size(zmin)-z0


# Load z-kernel vs redshift:
z_red,sig_z_red=np.loadtxt('z_kernel_5perc_ext_z01.txt', unpack=True) # 95% percentile;
z_kernel_size=ius(z_red, 1./100./np.sqrt(sig_z_red), k=1, ext=3)


In [ ]:
# Functions to compute \Sigma(R|lob,zob)


#########################################################################################
#### SIMGA_PRJ WITH SELECTION BIAS
min_mass4integral = 1.0e13
theta_grid_size=50
exclusion_sigma = True
#########################################################################################

def Sigma_prj_lobsel_CIRC(R, lob, zob, prj_depth=50.):
    """
    Compute the line-of-sight projected surface density Sigma(R) for the
    lob-selected sample, integrating over foreground and background structures.

    Parameters
    ----------
    R : array
        Projected cluster-centric radii at the cluster redshift [pMpc/h].
    lob : float
        Observed richness.
    zob : float
        Cluster redshift.
    prj_depth : float, optional
        Maximum comoving line-of-sight integration depth [cMpc/h].

    Returns
    -------
    integral : array
        Projected surface density Sigma(R) [M_sun h / pMpc^2].
    """
    # Line-of-sight distance grid (log-spaced, symmetric around the cluster)
    dis_max = prj_depth
    dis_min = 1.0e-6
    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)

    # Redshift grid for foreground structures
    ztr_grid = z_of_chi(chi_of_z(zob) - dis)[::-1]

    # Append background structures
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zob)))

    # Surface density contribution at each tracer redshift
    Sigma_mis_all_mass_red = np.array([
        int_over_ThetaRs_lobsel_CIRC(R, lob, zob, ztr_grid[i])
        for i in range(ztr_grid.size)
    ])  # [ztr_grid, R_grid] in M_sun h / pMpc^2

    # Weight by comoving volume element
    yarray = dVdzdOmg(ztr_grid)[:, None] * Sigma_mis_all_mass_red

    # Integrate over tracer redshift
    integral = np.trapz(yarray, ztr_grid, axis=0)
    return integral


def int_over_ThetaRs_lobsel_CIRC(R, lob, zob, ztr):
    """
    Integrate Sigma(R) over angular offsets between the projected halo
    and the point at radius R.

    Parameters
    ----------
    R : array
        Projected radii [pMpc/h].
    lob : float
        Observed richness.
    zob : float
        Cluster redshift.
    ztr : float
        Tracer redshift.

    Returns
    -------
    integral : array
        Surface density Sigma(R) [M_sun h / pMpc^2].
    """
    # Angular miscentering range (physical angles)
    lo = 1.0e-6
    hi = 30.  # maximum angular offset
    theta_grid = np.geomspace(lo, hi, theta_grid_size) / chi_of_z(zob)

    # Integrate over halo mass for each angular offset
    Sigma_mis_all_mass = np.array([
        int_over_M_lobsel_CIRC(R, lob, zob, ztr, theta_grid[i])
        for i in range(theta_grid.size)
    ])  # [theta_grid, R_grid]

    # Jacobian for angular integration
    yarray = np.sin(theta_grid)[:, None] * Sigma_mis_all_mass

    # Integrate over angular separation
    integral = np.trapz(yarray, theta_grid, axis=0)
    return integral


def int_over_M_lobsel_CIRC(R, lob, zob, ztr, theta_around_R):
    """
    Integrate Sigma(R) over the halo mass function.

    Parameters
    ----------
    R : array
        Projected radii [pMpc/h].
    lob : float
        Observed richness.
    zob : float
        Cluster redshift.
    ztr : float
        Tracer redshift.
    theta_around_R : float
        Angular separation between tracer halo and point R [rad].

    Returns
    -------
    integral : array
        Mass-integrated surface density [M_sun h / pMpc^2].
    """
    # Halo mass grid
    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 50)

    # Scale factor at cluster redshift
    a_scale = 1. / (1. + zob)

    # Concentration, HMF, and halo bias
    cs = c_of_m(m_grid, ztr)
    hmf = dndM(m_grid, ztr)               # [h^3 / cMpc^3]
    b_halo = bias_halo(m_grid, ztr)

    # Projected distance from tracer halo to point R
    R_theta_around_R_ztr = theta_around_R * chi_of_z(ztr)

    # NFW projected surface density (converted to physical units)
    Sigma_Rmis = (
        twoD_prj_NFW(R_theta_around_R_ztr, m_grid, cs, ztr)[0][0]
        / a_scale**2.
    )  # [M_sun h / pMpc^2]

    # Angular integration around the cluster center
    int_over_phi_CIRC_allR = np.array([
        int_over_phi_lobsel_CIRC(R[i], lob, zob, ztr, theta_around_R, b_halo)
        for i in range(R.size)
    ]).T  # [M_grid, R_grid]

    # Full mass integral
    yarray = (
        m_grid[:, None]
        * hmf[:, None]
        * Sigma_Rmis[:, None]
        * int_over_phi_CIRC_allR
    )

    integral = np.trapz(yarray, np.log(m_grid), axis=0)
    return integral


def int_over_phi_lobsel_CIRC(R, lob, zob, ztr, theta_around_R, b_halo, phi_grid_size=50):
    """
    Integrate the two-halo term over the azimuthal angle phi.

    Parameters
    ----------
    R : float
        Projected radius [pMpc/h].
    lob : float
        Observed richness.
    zob : float
        Cluster redshift.
    ztr : float
        Tracer redshift.
    theta_around_R : float
        Angular separation [rad].
    b_halo : array
        Halo bias as a function of mass.
    phi_grid_size : int
        Number of angular integration points.

    Returns
    -------
    integral : array
        Azimuthally integrated correlation term [M_grid].
    """
    phi_grid = np.linspace(0., np.pi, phi_grid_size)

    # Projected distance from cluster center
    R_theta_around_R = theta_around_R * chi_of_z(zob)
    Rtilde = np.sqrt(
        R_theta_around_R**2. + R**2.
        - 2. * R_theta_around_R * R * np.cos(phi_grid)
    )

    # Corresponding angular separation
    theta_Rtilde = Rtilde / chi_of_z(zob)

    # Effective selection bias
    eff_bias_lobztr_theta = b_sel_lob_theta_grid_inter(theta_Rtilde)

    # 3D separation between tracer and cluster
    dis = np.sqrt(
        chi_of_z(ztr)**2. + chi_of_z(zob)**2.
        - 2. * chi_of_z(ztr) * chi_of_z(zob) * np.cos(theta_Rtilde)
    )

    # Non-linear correlation function
    xi_NL = xir2_NL_r_z(np.log(dis), zob).T[0] / dis**2.

    # Two-halo contribution
    bM_bsel_xi = (
        b_halo[None, :]
        * eff_bias_lobztr_theta[:, None]
        * xi_NL[:, None]
    )

    # Exclusion region
    if exclusion_sigma:
        bM_bsel_xi[dis < R_lambda(lob) * (1. + zob), :] = -1.

    # Enforce lower bound
    bM_bsel_xi[bM_bsel_xi < -1.] = -1.

    yarray = 1. + bM_bsel_xi
    integral = 2. * np.trapz(yarray, phi_grid, axis=0)
    return integral


def twoD_prj_NFW(r, M, c, z):
    """
    Projected NFW surface density profile.

    Parameters
    ----------
    r : array
        Projected comoving radius [cMpc/h].
    M : array
        Halo mass M200m [Msun/h].
    c : array
        Halo concentration.
    z : float
        Redshift.

    Returns
    -------
    SigmaNFW : array
        Projected surface density [M_sun h / cMpc^2].
    """
    rho_m_z = Omega_m * rho_c

    # Halo radius and scale radius
    r200m = (M * 3. / (4. * np.pi * 200. * rho_m_z))**(1. / 3.)
    r_s = np.atleast_1d(r200m / c)

    # Dimensionless radius
    x = np.atleast_2d(r)[:, :, None] / r_s[None, None, :]

    # Characteristic overdensity
    f_c = np.log(1. + c) - c / (1. + c)
    delta_char = 200. / 3. * c**3. / f_c
    rho_s = np.atleast_1d(rho_m_z * delta_char)

    # NFW projected profile
    SigmaNFW = 2. * rho_s[None, None, :] * r_s[None, None, :] * fun(x)

    # Hard truncation at large radius
    arr_expanded = np.tile(
        np.expand_dims(r, axis=-1),
        (1, 1, np.atleast_1d(M).size)
    )
    R_max = 30.  # cMpc/h
    SigmaNFW = np.where(arr_expanded > R_max, 0., SigmaNFW)

    return SigmaNFW


def fun(x):
    """
    Auxiliary function for the projected NFW profile.

    Parameters
    ----------
    x : array
        Dimensionless radius r / r_s.

    Returns
    -------
    fun : array
        Analytical NFW projection kernel.
    """
    x = np.atleast_1d(x)
    fun = 1. - 2. / np.sqrt(np.abs(x**2. - 1.)) * np.arctan(
        np.sqrt((np.abs(x - 1.)) / (x + 1.))
    )

    fun[x < 1.] = (
        1. - 2. / np.sqrt(1. - x[x < 1.]**2.)
        * np.arctanh(np.sqrt((1. - x[x < 1.]) / (1. + x[x < 1.])))
    )

    fun[x != 1.] /= (x[x != 1.]**2. - 1.)
    fun[x == 1.] = 1. / 3.
    return fun


def Mprj_com(r, M, c):
    """
    Projected NFW mass enclosed within radius r (tau -> infinity).

    Parameters
    ----------
    r : float or array
        Projected radius [cMpc/h].
    M : float
        Halo mass M200m [Msun/h].
    c : float
        Concentration.

    Returns
    -------
    Mprj : array
        Projected mass [Msun/h].
    """
    rho_m_z = Omega_m * rho_c
    r200m = (M * 3. / (4. * np.pi * 200. * rho_m_z))**(1. / 3.)
    r_s = r200m / c

    x = r / r_s
    f_c = np.log(1. + c) - c / (1. + c)
    delta_char = 200. / 3. * c**3. / f_c
    rho_s = rho_m_z * delta_char
    M0 = rho_s * 4. * np.pi * r_s**3.

    effe_x = np.abs(np.arccos(1. / x + 0.j) / np.sqrt(x**2. - 1., dtype=complex))
    return M0 * (effe_x + np.log(x / 2.))


def c_of_m(M, z):
    """
    Mass–concentration relation for M200m halos.

    Parameters
    ----------
    M : array
        Halo mass [Msun/h].
    z : float
        Redshift.

    Returns
    -------
    c : array
        Concentration parameter.
    """
    A = 10.14
    B = -0.081
    C = -1.01
    Mpivot = 2. * 10.**12.  # Msun/h
    return A * (M / Mpivot)**B * (1 + z)**C


In [ ]:
# ============================================================
# Functions to compute optical cluster bias: b_lambda_ob
# ============================================================

# ------------------------------------------------------------
# Global parameters controlling the integrals and transitions
# ------------------------------------------------------------

min_mass4integral = 1.0e13   # Minimum halo mass [Msun/h] for mass integrals
damping_sigmoid = 2.5       # Steepness parameter of the sigmoid transition
n_th_inf = 0.0              # Lower bound (in units of theta_lob) for transition
n_th_sup = 1.0              # Upper bound (in units of theta_lob) for transition
exclusion = True            # Enable halo exclusion prescription


# ------------------------------------------------------------
# Simple memoization decorator for expensive integrals
# ------------------------------------------------------------

def memoize(func):
    """
    Memoization decorator to cache the output of expensive
    numerical functions as a function of their input arguments.

    Provides an additional method `.clear_cache()` to manually
    reset the stored cache.
    """
    cache = {}

    def memoized_func(*args):
        if args not in cache:
            result = func(*args)
            cache[args] = result
        return cache[args]
    
    def clear_cache():
        nonlocal cache
        cache = {}

    memoized_func.clear_cache = clear_cache
    return memoized_func


# ------------------------------------------------------------
# Scale-dependent optical selection bias
# ------------------------------------------------------------

@memoize
def b_sel_lob_ltr_theta(ltr, zcl, lob, theta, damping=damping_sigmoid):
    """
    Scale-dependent optical selection bias for a cluster with
    observed richness lob and true richness ltr.

    The bias interpolates smoothly between:
    - the intrinsic (small-scale) selection bias, and
    - the effective large-scale bias,
    using a sigmoid function in angular separation.

    Parameters
    ----------
    ltr : float
        True richness.
    zcl : float
        Cluster redshift.
    lob : float
        Observed richness.
    theta : array
        Angular separation from the cluster center [rad].
    damping : float
        Controls the steepness of the transition.

    Returns
    -------
    bsel_lobltr : array
        Scale-dependent selection bias.
    """

    # Angular size of the cluster aperture at the cluster redshift:
    # theta = R_lambda / D_A = R_lambda * (1 + z) / chi(z)
    theta_lobzob = R_lambda(lob) * (1. + zcl) / chi_of_z(zcl)

    # Intrinsic (small-scale) selection bias
    b_sel_lob_ltr_thetalob = b_sel_lob_ltr_in(ltr, zcl, lob)

    # Mean projected richness excess at fixed observed richness
    DPRJ = bar_delta_prj_Beff(lob, zcl)

    # Linear model for bias enhancement due to richness projection
    boost_bias = (
        inter_boost_bias
        + slope_boost_bias * (Delta_prj_lob_ltr(lob, ltr) - DPRJ) / DPRJ
    )

    # Effective large-scale bias
    bias_eff = eff_bias_ltr(ltr, zcl) * boost_bias

    # Sigmoid transition parameters
    x0 = (n_th_inf * theta_lobzob + n_th_sup * theta_lobzob) / 2.  # midpoint
    k = damping / (n_th_sup * theta_lobzob - n_th_inf * theta_lobzob)  # steepness

    # Smooth interpolation between small- and large-scale regimes
    bsel_lobltr = (
        b_sel_lob_ltr_thetalob
        + (bias_eff - b_sel_lob_ltr_thetalob)
        / (1. + np.exp(-k * (theta - x0)))
    )

    return bsel_lobltr


# ------------------------------------------------------------
# Redshift- and mass-integrated bias–xi kernel
# ------------------------------------------------------------

@memoize
def int_z_M_dV_n_bias_xi(theta, zcl):
    """
    Computes the redshift- and mass-integrated kernel entering
    the two-halo contribution to the optical selection bias.

    The kernel includes:
    - halo mass function
    - halo bias
    - comoving volume element
    - nonlinear matter correlation function

    The result is normalized by the same integral without bias
    and correlation terms.
    """

    # ------------------------
    # Foreground contribution
    # ------------------------

    z_min = bisect(zmin4zkernel, -2., 2., args=(zcl))
    z_max = zcl - 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    dis_max = chi_of_z(zcl) - chi_zmin
    dis_min = chi_of_z(zcl) - chi_zmax

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = z_of_chi(chi_of_z(zcl) - dis)[::-1]

    # ------------------------
    # Background contribution
    # ------------------------

    z_max = bisect(zmax4zkernel, -2., 2., args=(zcl))
    z_min = zcl + 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    dis_min = chi_zmin - chi_of_z(zcl)
    dis_max = chi_zmax - chi_of_z(zcl)

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zcl)))

    # ------------------------
    # Mass integration
    # ------------------------

    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 100)

    # Halo mass function and halo bias as functions of mass and redshift
    hmf = np.array([dndM(m_grid, ztr_grid[iz]) for iz in range(ztr_grid.size)]).T
    b_halo = np.array([bias_halo(m_grid, ztr_grid[iz]) for iz in range(ztr_grid.size)]).T

    # 3D comoving separation between cluster and line-of-sight halos
    dis = np.sqrt(
        chi_of_z(zcl)**2.
        + chi_of_z(ztr_grid)**2.
        - 2. * chi_of_z(zcl) * chi_of_z(ztr_grid) * np.cos(theta)
    )

    # Nonlinear matter correlation function
    xi_NL = np.array([
        xir2_NL_r_z(np.log(dis[i]), zcl)[0][0] / dis[i]**2.
        for i in range(dis.size)
    ])

    # Full biased integrand
    yarray = (
        m_grid[:, None]
        * b_halo
        * hmf
        * dVdzdOmg(ztr_grid)[None, :]
        * xi_NL[None, :]
    )

    # Integrate over mass and redshift
    integral_dlnM = np.trapz(yarray, np.log(m_grid), axis=0)
    integral_dz = np.trapz(integral_dlnM, ztr_grid, axis=0)

    # ------------------------
    # Normalization (no bias, no xi)
    # ------------------------

    yarray = m_grid[:, None] * hmf * dVdzdOmg(ztr_grid)[None, :]
    norm_integral_dlnM = np.trapz(yarray, np.log(m_grid), axis=0)
    norm_integral_dz = np.trapz(norm_integral_dlnM, ztr_grid, axis=0)

    return integral_dz / norm_integral_dz


# ------------------------------------------------------------
# Intrinsic (small-scale) selection bias
# ------------------------------------------------------------

@memoize
def b_sel_lob_ltr_in(ltr, zcl, lob):
    """
    Intrinsic optical selection bias, corresponding to the
    small-scale (theta -> 0) limit.

    Defined as the excess projected richness relative to the
    expected background contribution, normalized by the
    projected two-halo term.
    """

    # Mean projected richness excess
    DPRJ = bar_delta_prj_Beff(lob, zcl)

    # Linear boost model
    boost_bias = (
        inter_boost_bias
        + slope_boost_bias * (Delta_prj_lob_ltr(lob, ltr) - DPRJ) / DPRJ
    )

    # Background projected richness contribution
    barDeltaPRJ_BKG = bar_delta_prj_bkg(ltr, zcl, boost_bias)

    # Observed projected richness excess
    Delta_prj_lob = Delta_prj_lob_ltr(lob, ltr)

    # Two-halo normalization term
    numerator = numerator2(ltr, zcl)

    # Intrinsic selection bias
    bsel_lobltr = (Delta_prj_lob - barDeltaPRJ_BKG) / numerator

    return bsel_lobltr


# ------------------------------------------------------------
# Simple helper: projected richness excess
# ------------------------------------------------------------

def Delta_prj_lob_ltr(lob, ltr):
    """
    Projected richness excess.

    Parameters
    ----------
    lob : float
        Observed richness.
    ltr : float
        True richness.

    Returns
    -------
    Delta_prj : float
        lob - ltr
    """
    return lob - ltr


@memoize
def delta_prj_bkg(ltr, zcl):
    """
    Mean projected richness contribution from correlated structures
    along the line of sight (background + foreground).

    This quantity enters the intrinsic optical selection bias and
    represents the expectation value of projected richness due to
    halos correlated with the cluster but lying at different redshifts.

    Parameters
    ----------
    ltr : float
        True cluster richness.
    zcl : float
        Cluster redshift.

    Returns
    -------
    integral : float
        Background projected richness contribution.
        Units: [h^3 / cMpc^3]
    """

    # ------------------------
    # Foreground contribution
    # ------------------------

    z_min = bisect(zmin4zkernel, -2., 2., args=(zcl))
    z_max = zcl - 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Comoving distance interval in front of the cluster
    dis_max = chi_of_z(zcl) - chi_zmin
    dis_min = chi_of_z(zcl) - chi_zmax

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = z_of_chi(chi_of_z(zcl) - dis)[::-1]

    # ------------------------
    # Background contribution
    # ------------------------

    z_max = bisect(zmax4zkernel, -2., 2., args=(zcl))
    z_min = zcl + 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Comoving distance interval behind the cluster
    dis_min = chi_zmin - chi_of_z(zcl)
    dis_max = chi_zmax - chi_of_z(zcl)

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zcl)))

    # ------------------------
    # Mass-integrated contribution at each redshift
    # ------------------------

    intoverM = np.array([
        int_over_mass_DPRJ(zcl, ltr, ztr_grid[i])
        for i in range(ztr_grid.size)
    ])

    # Full redshift integrand: volume element × kernel × mass integral
    yarray = dVdzdOmg(ztr_grid) * kernel_z(ztr_grid, zcl) * intoverM

    # Integrate over redshift
    integral = np.trapz(yarray, ztr_grid, axis=0)

    # Assuming foreground and background contributions are treated symmetrically
    return integral  # [h^3 / cMpc^3]


def int_over_mass_DPRJ(zcl, lob, ztr):
    """
    Mass-integrated contribution to the projected richness background
    at a fixed tracer redshift.

    Parameters
    ----------
    zcl : float
        Cluster redshift.
    lob : float
        Observed richness of the cluster.
    ztr : float
        Tracer halo redshift.

    Returns
    -------
    integral : float
        Mass-integrated projected richness contribution.
        Units: [h^3 / cMpc^3]
    """

    # Halo mass grid
    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 50)

    # Halo mass function at tracer redshift
    hmf = dndM(m_grid, ztr)  # [h^3 / cMpc^3]

    # Integrand: halo mass × mass function × richness integral
    yarray = m_grid * hmf * int_over_lambda_DPRJ(lob, zcl, ztr, m_grid)

    # Integrate over log-mass
    integral = np.trapz(yarray, np.log(m_grid), axis=0)

    return integral  # [h^3 / cMpc^3]


def int_over_lambda_DPRJ(lob, zcl, z, M):
    """
    Integral over true richness entering the projected background
    contribution at fixed halo mass and redshift.

    This term accounts for:
    - the probability distribution of true richness at fixed mass,
    - the effective overlapping area,
    - the halo occupation probability.

    Parameters
    ----------
    lob : float
        Observed richness.
    zcl : float
        Cluster redshift.
    z : float
        Tracer halo redshift.
    M : array
        Halo mass grid [Msun/h].

    Returns
    -------
    integral : array
        Richness-integrated contribution as a function of halo mass.
    """

    # True richness grid (bounded by observed richness)
    ltr_grid = np.linspace(1.0e-10, lob, 100)

    # Effective occupation × overlapping area factor
    Om_f_lambsq = (
        Omega_halos(lob, zcl, ltr_grid, z)
        * f_area(lob, zcl, ltr_grid, z)
    )

    # Integrand: P(ltr | M, z) × ltr × occupation × area
    yarray = (
        pltr_M(ltr_grid[:, None], M[None, :], z)
        * ltr_grid[:, None]
        * Om_f_lambsq[:, None]
    )

    # Integrate over true richness
    integral = np.trapz(yarray, ltr_grid, axis=0)

    return integral

def bar_delta_prj_bkg(lob, zcl, boost_bias):
    """
    Mean projected richness background contribution entering the
    intrinsic optical selection bias, including clustering effects.

    This quantity accounts for the correlated foreground and background
    halos weighted by the effective cluster bias.

    Parameters
    ----------
    lob : float
        Observed cluster richness.
    zcl : float
        Cluster redshift.
    boost_bias : float
        Boost factor applied to the effective cluster bias.

    Returns
    -------
    integral : float
        Mean projected richness background contribution.
        Units: [h^3 / cMpc^3]
    """

    # ------------------------
    # Foreground contribution
    # ------------------------

    z_min = bisect(zmin4zkernel, -2., 2., args=(zcl))
    z_max = zcl - 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Comoving distance interval in front of the cluster
    dis_max = chi_of_z(zcl) - chi_zmin
    dis_min = chi_of_z(zcl) - chi_zmax

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = z_of_chi(chi_of_z(zcl) - dis)[::-1]

    # ------------------------
    # Background contribution
    # ------------------------

    z_max = bisect(zmax4zkernel, -2., 2., args=(zcl))
    z_min = zcl + 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Comoving distance interval behind the cluster
    dis_min = chi_zmin - chi_of_z(zcl)
    dis_max = chi_zmax - chi_of_z(zcl)

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zcl)))

    # ------------------------
    # Effective cluster bias
    # ------------------------

    bias_cl = eff_bias_ltr(lob, zcl) * boost_bias

    # ------------------------
    # Mass-integrated contribution at each redshift
    # ------------------------

    intoverM = np.array([
        int_over_mass2(zcl, lob, ztr_grid[i], bias_cl)
        for i in range(ztr_grid.size)
    ])

    # Full redshift integrand: volume × kernel × mass integral
    yarray = dVdzdOmg(ztr_grid) * kernel_z(ztr_grid, zcl) * intoverM

    # Integrate over redshift
    integral = np.trapz(yarray, ztr_grid, axis=0)

    # Foreground and background are assumed to contribute symmetrically
    return integral  # [h^3 / cMpc^3]


def int_over_mass2(zcl, lob, ztr, bias_cl):
    """
    Mass-integrated contribution to the projected richness background
    including halo bias terms.

    Parameters
    ----------
    zcl : float
        Cluster redshift.
    lob : float
        Observed richness.
    ztr : float
        Tracer halo redshift.
    bias_cl : float
        Effective cluster bias.

    Returns
    -------
    integral : float
        Mass-integrated projected richness contribution.
        Units: [h^3 / cMpc^3]
    """

    # Halo mass grid
    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 50)

    # Halo mass function and halo bias at tracer redshift
    hmf = dndM(m_grid, ztr)            # [h^3 / cMpc^3]
    bias_h = bias_halo(m_grid, ztr)

    # Integrand: mass × mass function × richness integral
    yarray = (
        m_grid
        * hmf
        * int_over_lambda2(lob, zcl, ztr, m_grid, bias_cl, bias_h)
    )

    # Integrate over log-mass
    integral = np.trapz(yarray, np.log(m_grid), axis=0)

    return integral  # [h^3 / cMpc^3]


def int_over_lambda2(lob, zcl, z, M, bias_cl, bias_h):
    """
    Integral over true richness including clustering corrections
    proportional to the halo–cluster correlation.

    Parameters
    ----------
    lob : float
        Observed richness.
    zcl : float
        Cluster redshift.
    z : float
        Tracer halo redshift.
    M : array
        Halo mass grid [Msun/h].
    bias_cl : float
        Effective cluster bias.
    bias_h : array
        Halo bias as a function of mass.

    Returns
    -------
    integral : array
        Richness-integrated contribution as a function of halo mass.
    """

    # True richness grid (bounded by observed richness)
    ltr_grid = np.linspace(1.0e-10, lob, 100)

    # Effective halo occupation × overlapping area factor
    Om_f_lambsq = (
        Omega_halos(lob, zcl, ltr_grid, z)
        * f_area(lob, zcl, ltr_grid, z)
    )

    # Integrand including clustering correction term
    yarray = (
        pltr_M(ltr_grid[:, None], M[None, :], z)
        * ltr_grid[:, None]
        * (
            Om_f_lambsq[:, None]
            + 2. * np.pi
            * bias_h[None, :]
            * bias_cl
            * int_over_phi2(lob, zcl, z, ltr_grid, bias_cl)[:, None]
        )
    )

    # Integrate over true richness
    integral = np.trapz(yarray, ltr_grid, axis=0)

    return integral


def int_over_phi2(lob, zcl, z, ltr_grid, bias_cl, damping=damping_sigmoid):
    """
    Angular integral encoding the clustering correction to the
    projected richness background.

    Includes:
    - nonlinear matter correlation function,
    - halo exclusion,
    - scale-dependent sigmoid transition,
    - geometric overlap between apertures.

    Parameters
    ----------
    lob : float
        Observed richness.
    zcl : float
        Cluster redshift.
    z : float
        Tracer halo redshift.
    ltr_grid : array
        True richness grid.
    bias_cl : float
        Effective cluster bias.
    damping : float
        Controls the steepness of the sigmoid transition.

    Returns
    -------
    integral : array
        Angularly integrated clustering correction.
    """

    # Angular radius of tracer and cluster apertures
    theta_ltrz = R_lambda(ltr_grid) * (1. + z) / chi_of_z(z)
    theta_lobzob = R_lambda(lob) * (1. + zcl) / chi_of_z(zcl)

    # Angular integration grid
    theta_grid = np.geomspace(1.0e-6, theta_lobzob + theta_ltrz, 50)

    # Sigmoid transition parameters
    x0 = (n_th_inf * theta_lobzob + n_th_sup * theta_lobzob) / 2.
    k = damping / (n_th_sup * theta_lobzob - n_th_inf * theta_lobzob)

    # 3D comoving separation
    dis = np.sqrt(
        chi_of_z(zcl)**2.
        + chi_of_z(z)**2.
        - 2. * chi_of_z(zcl) * chi_of_z(z) * np.cos(theta_grid.reshape(-1))
    )

    # Nonlinear matter correlation function
    xi_NL = np.array([
        xir2_NL_r_z(np.log(dis[i]), zcl)[0][0] / dis[i]**2.
        for i in range(dis.size)
    ])

    # Halo exclusion prescription
    if exclusion:
        xi_NL[dis < R_lambda(lob) * (1. + zcl)] = 0.
        # In principle, b*b*xi = -1 in the exclusion region

    # Full angular integrand
    yarray = (
        np.sin(theta_grid)
        * 1. / (1. + np.exp(-k * (theta_grid - x0)))
        * np.reshape(xi_NL, theta_grid.shape)
        * area_overlap(theta_grid, theta_lobzob, theta_ltrz)
    )

    # Integrate over angle
    integral = np.trapz(yarray, theta_grid, axis=0)

    return integral

def numerator2(lob, zcl):
    """
    Computes the numerator of the projected richness contribution by integrating
    over tracer redshift, halo mass, richness, and angular separation.

    Parameters
    ----------
    lob : float
        Observed richness of the cluster.
    zcl : float
        Redshift of the cluster.

    Returns
    -------
    integral : float
        Integrated quantity [h^3 / cMpc^3].
    """

    # --------------------------------------------------
    # FOREGROUND contribution (z_tr < z_cl)
    # --------------------------------------------------

    # Lower bound in redshift where kernel becomes negligible
    z_min = bisect(zmin4zkernel, -2., 2., args=(zcl))
    # Upper bound just below the cluster redshift
    z_max = zcl - 1.0e-5

    # Convert redshift limits to comoving distance
    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Line-of-sight distance range in front of the cluster
    dis_max = chi_of_z(zcl) - chi_zmin
    dis_min = chi_of_z(zcl) - chi_zmax

    # Log-spaced comoving distance grid
    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)

    # Convert distances back to tracer redshift (foreground)
    ztr_grid = z_of_chi(chi_of_z(zcl) - dis)[::-1]

    # --------------------------------------------------
    # BACKGROUND contribution (z_tr > z_cl)
    # --------------------------------------------------

    # Upper redshift bound where kernel becomes negligible
    z_max = bisect(zmax4zkernel, -2., 2., args=(zcl))
    # Lower bound just above the cluster redshift
    z_min = zcl + 1.0e-5

    # Convert redshift limits to comoving distance
    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    # Line-of-sight distance range behind the cluster
    dis_min = chi_zmin - chi_of_z(zcl)
    dis_max = chi_zmax - chi_of_z(zcl)

    # Log-spaced comoving distance grid
    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)

    # Append background tracer redshifts
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zcl)))

    # --------------------------------------------------
    # Integrate over halo mass at fixed tracer redshift
    # --------------------------------------------------

    intoverM = np.array([
        int_over_mass4(zcl, lob, ztr_grid[i])
        for i in range(ztr_grid.size)
    ])

    # Combine volume element, redshift kernel, and mass integral
    yarray = dVdzdOmg(ztr_grid) * kernel_z(ztr_grid, zcl) * intoverM

    # Integrate over tracer redshift
    integral = np.trapz(yarray, ztr_grid, axis=0)

    return integral  # [h^3 / cMpc^3]


def int_over_mass4(zcl, lob, ztr):
    """
    Integrates the contribution over halo mass at fixed tracer redshift.
    """

    # Halo mass grid
    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 100)

    # Halo mass function at tracer redshift
    hmf = dndM(m_grid, ztr)  # [h^3 / cMpc^3]

    # Integrand: mass-weighted halo bias times richness integral
    yarray = (
        m_grid
        * bias_halo(m_grid, ztr)
        * hmf
        * int_over_lambda4(lob, zcl, ztr, m_grid)
    )

    # Integrate over log mass
    integral = np.trapz(yarray, np.log(m_grid), axis=0)

    return integral  # [h^3 / cMpc^3]


def int_over_lambda4(lob, zcl, z, M):
    """
    Integrates over true richness (lambda_tr) at fixed halo mass and redshift.
    """

    # True richness grid
    ltr_grid = np.linspace(1.0e-10, lob, 100)

    # Angular integral multiplied by geometric factor
    Om_f_lambsq = ltr_grid * 2. * np.pi * int_over_phi4(lob, zcl, z, ltr_grid)

    # Combine with P(lambda_tr | M, z)
    yarray = Om_f_lambsq[:, None] * pltr_M(ltr_grid[:, None], M[None, :], z)

    # Integrate over true richness
    integral = np.trapz(yarray, ltr_grid, axis=0)

    return integral


def int_over_phi4(lob, zcl, z, ltr_grid, damping=damping_sigmoid):
    """
    Integrates the non-linear correlation function over angular separation.
    """

    # Angular radius corresponding to lambda_tr at tracer redshift
    theta_ltrz = R_lambda(ltr_grid) * (1. + z) / chi_of_z(z)

    # Angular radius corresponding to observed lambda at cluster redshift
    theta_lobzob = R_lambda(lob) * (1. + zcl) / chi_of_z(zcl)

    # Angular separation grid
    theta_grid = np.geomspace(1.0e-6, theta_lobzob + theta_ltrz, 50)

    # Sigmoid transition parameters
    x0 = (n_th_inf * theta_lobzob + n_th_sup * theta_lobzob) / 2.
    k = damping / (n_th_sup * theta_lobzob - n_th_inf * theta_lobzob)

    # 3D comoving separation
    dis = np.sqrt(
        chi_of_z(zcl)**2. +
        chi_of_z(z)**2. -
        2. * chi_of_z(zcl) * chi_of_z(z) * np.cos(theta_grid.reshape(-1))
    )

    # Non-linear correlation function divided by r^2
    xi_NL = np.array([
        xir2_NL_r_z(np.log(dis[i]), zcl)[0][0] / dis[i]**2.
        for i in range(dis.size)
    ])

    # Apply exclusion radius
    if exclusion:
        xi_NL[dis < R_lambda(lob) * (1. + zcl)] = 0.

    # Angular integrand
    yarray = (
        np.sin(theta_grid)
        * (1. - 1. / (1. + np.exp(-k * (theta_grid - x0))))
        * np.reshape(xi_NL, theta_grid.shape)
        * area_overlap(theta_grid, theta_lobzob, theta_ltrz)
    )

    # Integrate over angular separation
    integral = np.trapz(yarray, theta_grid, axis=0)

    return integral


# --------------------------------------------------
# Mean projected richness difference: <Delta_prj> = lob - ltr
# --------------------------------------------------

def bar_delta_prj_lob_ltr(lob, ltr, zcl):
    """
    Computes the mean projected richness difference lob - ltr.
    """

    # Foreground tracer redshifts
    z_min = bisect(zmin4zkernel, -2., 2., args=(zcl))
    z_max = zcl - 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    dis_max = chi_of_z(zcl) - chi_zmin
    dis_min = chi_of_z(zcl) - chi_zmax

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = z_of_chi(chi_of_z(zcl) - dis)[::-1]

    # Background tracer redshifts
    z_max = bisect(zmax4zkernel, -2., 2., args=(zcl))
    z_min = zcl + 1.0e-5

    chi_zmin = chi_of_z(z_min)
    chi_zmax = chi_of_z(z_max)

    dis_min = chi_zmin - chi_of_z(zcl)
    dis_max = chi_zmax - chi_of_z(zcl)

    dis = 10.**np.linspace(np.log10(dis_min), np.log10(dis_max), 50)
    ztr_grid = np.append(ztr_grid, z_of_chi(dis + chi_of_z(zcl)))

    # Selection-dependent cluster bias
    bias_cl = b_sel_lob_ltr_in(ltr, zcl, lob)

    # Integrate over halo mass
    intoverM = np.array([
        int_over_mass5(zcl, ltr, ztr_grid[i], bias_cl)
        for i in range(ztr_grid.size)
    ])

    yarray = dVdzdOmg(ztr_grid) * kernel_z(ztr_grid, zcl) * intoverM
    integral = np.trapz(yarray, ztr_grid, axis=0)

    return integral  # [h^3 / cMpc^3]


def int_over_mass5(zcl, lob, ztr, bias_cl):
    """
    Mass integral including halo bias and cluster selection bias.
    """

    m_grid = 10.**np.linspace(np.log10(min_mass4integral), 15.5, 100)
    hmf = dndM(m_grid, ztr)
    bias_h = bias_halo(m_grid, ztr)

    yarray = (
        m_grid
        * hmf
        * int_over_lambda5(lob, zcl, ztr, m_grid, bias_cl, bias_h)
    )

    integral = np.trapz(yarray, np.log(m_grid), axis=0)

    return integral


def int_over_lambda5(lob, zcl, z, M, bias_cl, bias_h):
    """
    Integrates over true richness including correlated LSS term.
    """

    ltr_grid = np.linspace(1.0e-10, lob, 100)

    Om_f_lambsq = Omega_halos(lob, zcl, ltr_grid, z) * f_area(lob, zcl, ltr_grid, z)

    yarray = (
        pltr_M(ltr_grid[:, None], M[None, :], z)
        * ltr_grid[:, None]
        * (
            Om_f_lambsq[:, None]
            + 2. * np.pi
            * bias_h[None, :]
            * int_over_phi5(
                lob, zcl, z, ltr_grid, bias_cl, np.mean(bias_h)
            )[:, None]
        )
    )

    integral = np.trapz(yarray, ltr_grid, axis=0)

    return integral


def int_over_phi5(lob, zcl, z, ltr_grid, bias_cl, mean_bias_h, damping=damping_sigmoid):
    """
    Angular integral with smooth transition between cluster and LSS bias regimes.
    """

    theta_ltrz = R_lambda(ltr_grid) * (1. + z) / chi_of_z(z)
    theta_lobzob = R_lambda(lob) * (1. + zcl) / chi_of_z(zcl)

    theta_grid = np.geomspace(1.0e-6, theta_lobzob + theta_ltrz, 50)

    x0 = (n_th_inf * theta_lobzob + n_th_sup * theta_lobzob) / 2.
    k = damping / (n_th_sup * theta_lobzob - n_th_inf * theta_lobzob)

    dis = np.sqrt(
        chi_of_z(zcl)**2. +
        chi_of_z(z)**2. -
        2. * chi_of_z(zcl) * chi_of_z(z) * np.cos(theta_grid.reshape(-1))
    )

    xi_NL = np.array([
        xir2_NL_r_z(np.log(dis[i]), zcl)[0][0] / dis[i]**2.
        for i in range(dis.size)
    ])

    if exclusion:
        xi_NL[dis < R_lambda(lob) * (1. + zcl)] = 0.

    yarray = (
        np.sin(theta_grid)
        * (
            bias_cl
            + (eff_bias_ltr(lob, zcl) - bias_cl)
            / (1. + np.exp(-k * (theta_grid - x0)))
        )
        * np.reshape(xi_NL, theta_grid.shape)
        * area_overlap(theta_grid, theta_lobzob, theta_ltrz)
    )

    integral = np.trapz(yarray, theta_grid, axis=0)

    return integral


def plob_ltr(ltr,ztr,lob):
    '''
    P(lob|ltr,z)=(1-f_prj(ltr,z)) \delta_D(lob-ltr) + f_prj*tau e^[-tau(lob-ltr)] \Theta_H(lob-ltr)
    '''
    ltrs=np.atleast_1d(ltr)
    mean_dpr = bar_delta_prj_Beff(lob,zcl)
    var_dpr = var_delta_prj_Beff(lob,zcl)
    tau = 2. * mean_dpr / (mean_dpr**2. + var_dpr)
    f_prj=2. * mean_dpr**2. / (mean_dpr**2. + var_dpr)
    f_prj = min(1.0,f_prj)
    pdf = f_prj*tau*np.exp(-tau*(lob-ltrs))
    pdf[lob-ltr==np.amin(lob-ltr)]+= (1.-f_prj)
    pdf[lob<ltr]=0.
    return pdf

def d_plob_Mz(ltr,lob,ztr,mass):
    return plob_ltr(ltr,ztr, lob)*pltr_M(ltr,mass,ztr)

def plob_Mz(lob,mass,ztr):
    '''
    P(lob|M,z)=(1-f_prj(ltr=lob))*P(ltr=lob|M,z)+\int d ltr f_prj(ltr)*P(lob|ltr,z)*P(ltr|M,z)
    '''
    inf_int=1.0e-10
    barDeltaPRJ_BKG=0. #np.float64(bar_delta_prj_bkg(lob,ztr))
    sup_int=lob+barDeltaPRJ_BKG
    ltr_grid=np.linspace(inf_int,sup_int,100)
    y_array=d_plob_Mz(ltr_grid,lob,ztr,mass)
    pdf=np.trapz(y_array,ltr_grid,axis=0)+(1.-fprj_model(lob,afprj(ztr),bfprj(ztr)))*pltr_M(lob,mass,ztr)
    return pdf

# Effective bias: b_eff(lob,z) = \int dm b(M,z) n(M,z) P(lob|M,z) / \int dm n(M,z) P(lob|M,z)
# Note: assumed ztr=zob
@memoize
def eff_bias(lob,ztr):
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,100)
    hmf=dndM(m_grid,ztr)
    PlobMz=np.array([plob_Mz(lob,m_grid[i],ztr) for i in range(m_grid.size)])#plob_Mz(lob,m_grid,ztr)
    normalization=np.trapz(hmf*PlobMz*m_grid,np.log(m_grid))
    b_halo = bias_halo(m_grid,ztr)
    return np.trapz(b_halo*hmf*PlobMz*m_grid,np.log(m_grid))/normalization

# Effective bias: b_eff(ltr,z) = \int dm b(M,z) n(M,z) P(ltr|M,z) / \int dm n(M,z) P(ltr|M,z)
# Note: assumed ztr=zob
@memoize
def eff_bias_ltr(ltr,ztr):
#     return 0.
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,100)
    hmf=dndM(m_grid,ztr)
    PltrMz=pltr_M(ltr,m_grid,ztr)
    normalization=np.trapz(hmf*PltrMz*m_grid,np.log(m_grid))
    b_halo = bias_halo(m_grid,ztr)
    return np.trapz(b_halo*hmf*PltrMz*m_grid,np.log(m_grid))/normalization

@memoize
def bar_delta_prj_Beff(lob,zcl,boost_bias=1.0):
    # FOREGROUND OF THE CLUSTER
    z_min=bisect(zmin4zkernel,-2.,2.,args=(zcl))
    z_max=zcl-1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_max=chi_of_z(zcl)-chi_zmin
    dis_min=chi_of_z(zcl)-chi_zmax
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=z_of_chi(chi_of_z(zcl)-dis)[::-1]
    # BACKGROUND OF THE CLUSTER
    z_max=bisect(zmax4zkernel,-2.,2.,args=(zcl))
    z_min=zcl+1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_min=chi_zmin-chi_of_z(zcl)
    dis_max=chi_zmax-chi_of_z(zcl)
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=np.append(ztr_grid,z_of_chi(dis+chi_of_z(zcl)))
    bias_cl=eff_bias_ltr(lob,zcl)*boost_bias
    intoverM=np.array([int_over_mass_Beff(zcl,lob,ztr_grid[i],bias_cl) for i in range(ztr_grid.size)])
    yarray=dVdzdOmg(ztr_grid)*kernel_z(ztr_grid,zcl)*intoverM
    integral=np.trapz(yarray,ztr_grid,axis=0)
    return integral # [h^3/cMpc^3]  
  
def int_over_mass_Beff(zcl,lob,ztr,bias_cl): 
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,50)
    hmf=dndM(m_grid,ztr) # [h^3/cMpc^3]
    bias_h=bias_halo(m_grid,ztr)
    yarray = m_grid*hmf*int_over_lambda_Beff(lob,zcl,ztr,m_grid,bias_cl,bias_h) # [Mgrid,zgrid]
    integral=np.trapz(yarray,np.log(m_grid),axis=0)
    return integral # [h^3/cMpc^3]

def int_over_lambda_Beff(lob,zcl,z,M,bias_cl,bias_h):
    ltr_grid=np.linspace(1.0e-10,lob,100)
    yarray=pltr_M(ltr_grid[:,None],M[None,:],z)*ltr_grid[:,None]*(
                    2.*np.pi*int_over_phi_Beff(lob,zcl,z,ltr_grid,bias_cl,bias_h))
    integral=np.trapz(yarray,ltr_grid,axis=0)
    return integral

def int_over_phi_Beff(lob,zcl,z,ltr_grid,bias_cl,bias_h,damping=damping_sigmoid):
    theta_ltrz=R_lambda(ltr_grid) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zcl)/chi_of_z(zcl) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_grid=np.geomspace(1.0e-6,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    dis=np.sqrt(chi_of_z(zcl)**2.+chi_of_z(z)**2.-2.*chi_of_z(zcl)*chi_of_z(z)*np.cos(theta_grid.reshape(-1))) # [theta,ltr]
    xi_NL = np.array([xir2_NL_r_z(np.log(dis[i]),zcl)[0][0]/dis[i]**2. for i in range(dis.size)])
    b0,b1,b2=[2.0,1./2.,0.75]
    theta_1Mpc= R_lambda(lob)/chi_of_z(zob)
    x0 = b0*theta_1Mpc
    k = b1*10. / x0  # steepness of the transition
    eff_bias_lobztr_theta=(bias_cl*b2+(bias_cl-bias_cl*b2)/(1.+np.exp(-k*(theta_grid-x0)))) # [theta,ltr]
    bM_bsel_xi=bias_h[None,None,:]*eff_bias_lobztr_theta[:,:,None]*np.reshape(xi_NL,theta_grid.shape)[:,:,None] # [theta_grid,ltr,Mgrid]
    if exclusion_sigma: bM_bsel_xi[np.reshape(dis,theta_grid.shape)<R_lambda(lob) *(1.+zob),:] = -1.
    bM_bsel_xi[bM_bsel_xi<-1.]=-1
    yarray=np.sin(theta_grid)[:,:,None]*(1.+bM_bsel_xi)*area_overlap(theta_grid,theta_lobzob,theta_ltrz)[:,:,None]   
    integral=np.trapz(yarray,theta_grid[:,:,None],axis=0) # [ltr,Mgrid]
    return integral

@memoize
def var_delta_prj_Beff(lob,zcl,boost_bias=1.0):
    # FOREGROUND OF THE CLUSTER
    z_min=bisect(zmin4zkernel,-2.,2.,args=(zcl))
    z_max=zcl-1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_max=chi_of_z(zcl)-chi_zmin
    dis_min=chi_of_z(zcl)-chi_zmax
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=z_of_chi(chi_of_z(zcl)-dis)[::-1]
    # BACKGROUND OF THE CLUSTER
    z_max=bisect(zmax4zkernel,-2.,2.,args=(zcl))
    z_min=zcl+1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_min=chi_zmin-chi_of_z(zcl)
    dis_max=chi_zmax-chi_of_z(zcl)
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=np.append(ztr_grid,z_of_chi(dis+chi_of_z(zcl)))
    bias_cl=eff_bias_ltr(lob,zcl)*boost_bias
    intoverM=np.array([int_over_mass_varBeff(zcl,lob,ztr_grid[i],bias_cl) for i in range(ztr_grid.size)])
    yarray=dVdzdOmg(ztr_grid)*kernel_z(ztr_grid,zcl)**2.*intoverM
    integral=np.trapz(yarray,ztr_grid,axis=0)
    return integral # [h^3/cMpc^3]  
  
def int_over_mass_varBeff(zcl,lob,ztr,bias_cl): 
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,50)
    hmf=dndM(m_grid,ztr) # [h^3/cMpc^3]
    bias_h=bias_halo(m_grid,ztr)
    yarray = m_grid*hmf*int_over_lambda_varBeff(lob,zcl,ztr,m_grid,bias_cl,bias_h) # [Mgrid,zgrid]
    integral=np.trapz(yarray,np.log(m_grid),axis=0)
    return integral # [h^3/cMpc^3]

def int_over_lambda_varBeff(lob,zcl,z,M,bias_cl,bias_h):
    ltr_grid=np.linspace(1.0e-10,lob,100)
    yarray=pltr_M(ltr_grid[:,None],M[None,:],z)*ltr_grid[:,None]**2.*(
                    2.*np.pi*int_over_phi_varBeff(lob,zcl,z,ltr_grid,bias_cl,bias_h))
    integral=np.trapz(yarray,ltr_grid,axis=0)
    return integral

def int_over_phi_varBeff(lob,zcl,z,ltr_grid,bias_cl,bias_h,damping=damping_sigmoid):
    theta_ltrz=R_lambda(ltr_grid) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zcl)/chi_of_z(zcl) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
#     theta_grid=np.linspace(0.,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    theta_grid=np.geomspace(1.0e-6,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    dis=np.sqrt(chi_of_z(zcl)**2.+chi_of_z(z)**2.-2.*chi_of_z(zcl)*chi_of_z(z)*np.cos(theta_grid.reshape(-1))) # [theta,ltr]
    xi_NL = np.array([xir2_NL_r_z(np.log(dis[i]),zcl)[0][0]/dis[i]**2. for i in range(dis.size)])
    b0,b1,b2=[2.0,1./2.,0.75]
    theta_1Mpc= R_lambda(lob)/chi_of_z(zob)
    x0 = b0*theta_1Mpc
    k = b1*10. / x0  # steepness of the transition
    eff_bias_lobztr_theta=(bias_cl*b2+(bias_cl-bias_cl*b2)/(1.+np.exp(-k*(theta_grid-x0)))) # [theta,ltr]
    bM_bsel_xi=bias_h[None,None,:]*eff_bias_lobztr_theta[:,:,None]*np.reshape(xi_NL,theta_grid.shape)[:,:,None] # [theta_grid,ltr,Mgrid]
    if exclusion_sigma: bM_bsel_xi[np.reshape(dis,theta_grid.shape)<R_lambda(lob) *(1.+zob),:] = -1.
    bM_bsel_xi[bM_bsel_xi<-1.]=-1
    yarray=np.sin(theta_grid)[:,:,None]*(1.+bM_bsel_xi)*(area_overlap(theta_grid,theta_lobzob,theta_ltrz)[:,:,None])**2.   
    integral=np.trapz(yarray,theta_grid[:,:,None],axis=0) # [ltr,Mgrid]
    return integral

def area_overlap(theta, theta_lob, theta_l):
    """
    Fractional overlap area between two circular apertures in projection.

    Parameters
    ----------
    theta : array
        Angular separation between halo centers [rad].
    theta_lob : float
        Angular radius of the main halo.
    theta_l : array
        Angular radius of the secondary halo.

    Returns
    -------
    A_overlap : array
        Fraction of overlapping area.
    """
    theta_l_reshape = theta_l[None, :] * np.ones(theta.shape)

    # Initialize full overlap
    A_overlap = np.ones(theta.shape)

    # No overlap
    A_overlap[theta > theta_lob + theta_l_reshape] = 0.

    # Full containment
    mask = theta_l_reshape > theta_lob
    A_overlap[mask] = theta_lob**2. / theta_l_reshape[mask]**2.

    # Partial overlap
    cond = theta > np.abs(theta_lob - theta_l_reshape)

    argcos1 = (
        (theta[cond]**2. + theta_l_reshape[cond]**2. - theta_lob**2.)
        / (2. * theta[cond] * theta_l_reshape[cond])
    )
    argcos1 = np.clip(argcos1, -1., 1.)

    argcos2 = (
        (theta[cond]**2. + theta_lob**2. - theta_l_reshape[cond]**2.)
        / (2. * theta[cond] * theta_lob)
    )
    argcos2 = np.clip(argcos2, -1., 1.)

    argsqrt = (
        (-theta[cond] + theta_l_reshape[cond] + theta_lob)
        * (theta[cond] + theta_l_reshape[cond] - theta_lob)
        * (theta[cond] - theta_l_reshape[cond] + theta_lob)
        * (theta[cond] + theta_l_reshape[cond] + theta_lob)
    )
    argsqrt[argsqrt < 0.] = 0.

    A_overlap[cond] = (
        theta_l_reshape[cond]**2. * np.arccos(argcos1)
        + theta_lob**2. * np.arccos(argcos2)
        - 0.5 * np.sqrt(argsqrt)
    ) / (np.pi * theta_l_reshape[cond]**2.)

    return A_overlap


In [ ]:
### Load and join richness and Sigma profile mock catalogs 
table_lob_sigma= Table.read('mock_lob_sigma_catalog.fits', format='fits')

catalog_z=table_lob_sigma['Z']
catalog_ra=table_lob_sigma['RA']
catalog_dec=table_lob_sigma['DEC']
catalog_m200b=table_lob_sigma['M200'] # [M_sun/h] wrt the mean matter density
catalog_lambda=table_lob_sigma['LAMBDA_TR_LOB']
catalog_lambda_tot=table_lob_sigma['LAMBDA_OB_LOB']
Sigma_R=table_lob_sigma['SIGMA_of_R']
Sigma_R_PRJ=table_lob_sigma['SIGMA_PRJ_of_R']
del table_lob_sigma

z_grid=np.linspace(np.amin(catalog_z),np.amax(catalog_z)+0.01,1000)
d_a_4interp=[d_a(z_grid[i],Omega_m) for i in range(len(z_grid))]
d_a_4interp=np.array(d_a_4interp)
d_a_interp=ius(z_grid,d_a_4interp,k=1)
catalog_d_a=d_a_interp(catalog_z)
catalog_theta_lob=np.degrees(R_lambda(catalog_lambda_tot)/catalog_d_a)
catalog_theta_ltr=np.degrees(R_lambda(catalog_lambda)/catalog_d_a)

In [ ]:
# Function to compute the stacked density profiles weighted by target mass and redshift distributions
# Credits: Hao-Yi Wu

def stacked_profile_weighted_by_mass_redshift(lnM_select, z_select, profile_select, lnM_all,
                                              z_all, profile_all, dm=0.1, dz=0.05):
    #### set up the bins for mass and redshift
    min_m = min(lnM_select)#-dm
    max_m = max(lnM_select)#+dm
    min_z = min(z_select)#-dz
    max_z = max(z_select)#+dz

#     print('z', min_z, max_z)
    m_bins = np.arange(min_m, max_m+dm, dm)
    z_bins = np.arange(min_z, max_z+dz, dz)
    nM = len(m_bins)-1
    nz = len(z_bins)-1
#     print('nM, nz', nM, nz)

    nr = np.shape(profile_select)[1]#rbp.nbins_phys_mpc
    profile_weighted = np.zeros(nr)
    weight_norm = 0


    pdf1_list = np.zeros([nz, nM]) # see how many bins are too narrow
    pdf2_list = np.zeros([nz, nM]) # see how many bins are too narrow

    for iz in range(nz):
        z_lo = z_bins[iz]
        z_hi = z_bins[iz+1]
        for iM in range(nM):
            m_lo = m_bins[iM]
            m_hi = m_bins[iM+1]
            select_bin = (lnM_select >= m_lo)&(lnM_select < m_hi)&(z_select>=z_lo)&(z_select<z_hi)
            weight = len(lnM_select[select_bin]) * 1.
            weight_norm += weight
            select_all = (lnM_all >= m_lo)&(lnM_all < m_hi)&(z_all>=z_lo)&(z_all<z_hi)

            pdf1_list[iz, iM] = weight
            pdf2_list[iz, iM] = len(lnM_all[select_all])


            if weight > 0 and len(lnM_all[select_all]) > 0:
                profile_weighted += (np.mean(profile_all[select_all, :], axis=0)*weight)

    profile_weighted /= weight_norm

    pdf1_list = np.concatenate(pdf1_list)
    pdf2_list = np.concatenate(pdf2_list)
    diff_list = pdf2_list - pdf1_list
    return profile_weighted





def stacked_profile_weighted_by_mass_redshift_std(lnM_select, z_select, profile_select, lnM_all,
                                              z_all, profile_all, dm=0.1, dz=0.05):
    #### set up the bins for mass and redshift
    min_m = min(lnM_select)#-dm
    max_m = max(lnM_select)#+dm
    min_z = min(z_select)#-dz
    max_z = max(z_select)#+dz

    m_bins = np.arange(min_m, max_m+dm, dm)
    z_bins = np.arange(min_z, max_z+dz, dz)
    nM = len(m_bins)-1
    nz = len(z_bins)-1

    nr = np.shape(profile_select)[1]#rbp.nbins_phys_mpc
    std_profile_weighted = np.zeros(nr)
    profile_weighted = np.zeros(nr)
    weight_norm = 0
    sum_std_weight_sq = 0.

    for iz in range(nz):
        z_lo = z_bins[iz]
        z_hi = z_bins[iz+1]
        for iM in range(nM):
            m_lo = m_bins[iM]
            m_hi = m_bins[iM+1]
            select_bin = (lnM_select >= m_lo)&(lnM_select < m_hi)&(z_select>=z_lo)&(z_select<z_hi)
            weight = len(lnM_select[select_bin]) * 1.
            weight_norm += weight
            select_all = (lnM_all >= m_lo)&(lnM_all < m_hi)&(z_all>=z_lo)&(z_all<z_hi)

            if weight > 0 and len(lnM_all[select_all]) > 0:
                profile_weighted += (np.mean(profile_all[select_all, :], axis=0)*weight)
                std_mean = np.std(profile_all[select_all, :], axis=0)/np.sqrt(lnM_all[select_all].size)
                std_weight= np.sqrt(weight) 
                std_profile_weighted += weight**2.*std_mean**2. + np.mean(profile_all[select_all, :], axis=0)**2.*std_weight
                sum_std_weight_sq += std_weight**2.

    profile_weighted /= weight_norm
    std_profile_weighted += profile_weighted**2.*sum_std_weight_sq          
    std_profile_weighted /= weight_norm**2.

    return np.sqrt(std_profile_weighted)


def plob_ltr(ltr,ztr,lob):
    '''
    P(lob|ltr,z)=(1-f_prj(ltr,z)) \delta_D(lob-ltr) + f_prj*tau e^[-tau(lob-ltr)] \Theta_H(lob-ltr)
    '''
    ltrs=np.atleast_1d(ltr)
    mean_dpr = bar_delta_prj_Beff(lob,zcl)
    var_dpr = var_delta_prj_Beff(lob,zcl)
    tau = 2. * mean_dpr / (mean_dpr**2. + var_dpr)
    f_prj=2. * mean_dpr**2. / (mean_dpr**2. + var_dpr)
    f_prj = min(1.0,f_prj)
    pdf = f_prj*tau*np.exp(-tau*(lob-ltrs))
    pdf[lob-ltr==np.amin(lob-ltr)]+= (1.-f_prj)
    pdf[lob<ltr]=0.
    return pdf

def d_plob_Mz(ltr,lob,ztr,mass):
    return plob_ltr(ltr,ztr, lob)*pltr_M(ltr,mass,ztr)

def plob_Mz(lob,mass,ztr):
    '''
    P(lob|M,z)=(1-f_prj(ltr=lob))*P(ltr=lob|M,z)+\int d ltr f_prj(ltr)*P(lob|ltr,z)*P(ltr|M,z)
    '''
    inf_int=1.0e-10
    barDeltaPRJ_BKG=0. #np.float64(bar_delta_prj_bkg(lob,ztr))
    sup_int=lob+barDeltaPRJ_BKG
    ltr_grid=np.linspace(inf_int,sup_int,100)
    y_array=d_plob_Mz(ltr_grid,lob,ztr,mass)
    pdf=np.trapz(y_array,ltr_grid,axis=0)+(1.-fprj_model(lob,afprj(ztr),bfprj(ztr)))*pltr_M(lob,mass,ztr)
    return pdf

# Effective bias: b_eff(lob,z) = \int dm b(M,z) n(M,z) P(lob|M,z) / \int dm n(M,z) P(lob|M,z)
# Note: assumed ztr=zob
@memoize
def eff_bias(lob,ztr):
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,100)
    hmf=dndM(m_grid,ztr)
    PlobMz=np.array([plob_Mz(lob,m_grid[i],ztr) for i in range(m_grid.size)])#plob_Mz(lob,m_grid,ztr)
    normalization=np.trapz(hmf*PlobMz*m_grid,np.log(m_grid))
    b_halo = bias_halo(m_grid,ztr)
    return np.trapz(b_halo*hmf*PlobMz*m_grid,np.log(m_grid))/normalization

# Effective bias: b_eff(ltr,z) = \int dm b(M,z) n(M,z) P(ltr|M,z) / \int dm n(M,z) P(ltr|M,z)
# Note: assumed ztr=zob
@memoize
def eff_bias_ltr(ltr,ztr):
#     return 0.
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,100)
    hmf=dndM(m_grid,ztr)
    PltrMz=pltr_M(ltr,m_grid,ztr)
    normalization=np.trapz(hmf*PltrMz*m_grid,np.log(m_grid))
    b_halo = bias_halo(m_grid,ztr)
    return np.trapz(b_halo*hmf*PltrMz*m_grid,np.log(m_grid))/normalization

@memoize
def bar_delta_prj_Beff(lob,zcl,boost_bias=1.0):
    # FOREGROUND OF THE CLUSTER
    z_min=bisect(zmin4zkernel,-2.,2.,args=(zcl))
    z_max=zcl-1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_max=chi_of_z(zcl)-chi_zmin
    dis_min=chi_of_z(zcl)-chi_zmax
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=z_of_chi(chi_of_z(zcl)-dis)[::-1]
    # BACKGROUND OF THE CLUSTER
    z_max=bisect(zmax4zkernel,-2.,2.,args=(zcl))
    z_min=zcl+1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_min=chi_zmin-chi_of_z(zcl)
    dis_max=chi_zmax-chi_of_z(zcl)
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=np.append(ztr_grid,z_of_chi(dis+chi_of_z(zcl)))
    bias_cl=eff_bias_ltr(lob,zcl)*boost_bias
    intoverM=np.array([int_over_mass_Beff(zcl,lob,ztr_grid[i],bias_cl) for i in range(ztr_grid.size)])
    yarray=dVdzdOmg(ztr_grid)*kernel_z(ztr_grid,zcl)*intoverM
    integral=np.trapz(yarray,ztr_grid,axis=0)
    return integral # [h^3/cMpc^3]  
  
def int_over_mass_Beff(zcl,lob,ztr,bias_cl): 
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,50)
    hmf=dndM(m_grid,ztr) # [h^3/cMpc^3]
    bias_h=bias_halo(m_grid,ztr)
    yarray = m_grid*hmf*int_over_lambda_Beff(lob,zcl,ztr,m_grid,bias_cl,bias_h) # [Mgrid,zgrid]
    integral=np.trapz(yarray,np.log(m_grid),axis=0)
    return integral # [h^3/cMpc^3]

def int_over_lambda_Beff(lob,zcl,z,M,bias_cl,bias_h):
    ltr_grid=np.linspace(1.0e-10,lob,100)
    yarray=pltr_M(ltr_grid[:,None],M[None,:],z)*ltr_grid[:,None]*(
                    2.*np.pi*int_over_phi_Beff(lob,zcl,z,ltr_grid,bias_cl,bias_h))
    integral=np.trapz(yarray,ltr_grid,axis=0)
    return integral

def int_over_phi_Beff(lob,zcl,z,ltr_grid,bias_cl,bias_h,damping=damping_sigmoid):
    theta_ltrz=R_lambda(ltr_grid) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zcl)/chi_of_z(zcl) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
#     theta_grid=np.linspace(0.,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    theta_grid=np.geomspace(1.0e-6,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    dis=np.sqrt(chi_of_z(zcl)**2.+chi_of_z(z)**2.-2.*chi_of_z(zcl)*chi_of_z(z)*np.cos(theta_grid.reshape(-1))) # [theta,ltr]
    xi_NL = np.array([xir2_NL_r_z(np.log(dis[i]),zcl)[0][0]/dis[i]**2. for i in range(dis.size)])
    b0,b1,b2=[2.0,1./2.,0.75]
    theta_1Mpc= R_lambda(lob)/chi_of_z(zob)
    x0 = b0*theta_1Mpc
    k = b1*10. / x0  # steepness of the transition
    eff_bias_lobztr_theta=(bias_cl*b2+(bias_cl-bias_cl*b2)/(1.+np.exp(-k*(theta_grid-x0)))) # [theta,ltr]
    bM_bsel_xi=bias_h[None,None,:]*eff_bias_lobztr_theta[:,:,None]*np.reshape(xi_NL,theta_grid.shape)[:,:,None] # [theta_grid,ltr,Mgrid]
    if exclusion_sigma: bM_bsel_xi[np.reshape(dis,theta_grid.shape)<R_lambda(lob) *(1.+zob),:] = -1.
    bM_bsel_xi[bM_bsel_xi<-1.]=-1
    yarray=np.sin(theta_grid)[:,:,None]*(1.+bM_bsel_xi)*area_overlap(theta_grid,theta_lobzob,theta_ltrz)[:,:,None]   
    integral=np.trapz(yarray,theta_grid[:,:,None],axis=0) # [ltr,Mgrid]
    return integral

@memoize
def var_delta_prj_Beff(lob,zcl,boost_bias=1.0):
    # FOREGROUND OF THE CLUSTER
    z_min=bisect(zmin4zkernel,-2.,2.,args=(zcl))
    z_max=zcl-1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_max=chi_of_z(zcl)-chi_zmin
    dis_min=chi_of_z(zcl)-chi_zmax
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=z_of_chi(chi_of_z(zcl)-dis)[::-1]
    # BACKGROUND OF THE CLUSTER
    z_max=bisect(zmax4zkernel,-2.,2.,args=(zcl))
    z_min=zcl+1.0e-5
    chi_zmin=chi_of_z(z_min)
    chi_zmax=chi_of_z(z_max)
    dis_min=chi_zmin-chi_of_z(zcl)
    dis_max=chi_zmax-chi_of_z(zcl)
    dis = 10.**np.linspace(np.log10(dis_min),np.log10(dis_max),50)
    ztr_grid=np.append(ztr_grid,z_of_chi(dis+chi_of_z(zcl)))
    bias_cl=eff_bias_ltr(lob,zcl)*boost_bias
    intoverM=np.array([int_over_mass_varBeff(zcl,lob,ztr_grid[i],bias_cl) for i in range(ztr_grid.size)])
    yarray=dVdzdOmg(ztr_grid)*kernel_z(ztr_grid,zcl)**2.*intoverM
    integral=np.trapz(yarray,ztr_grid,axis=0)
    return integral # [h^3/cMpc^3]  
  
def int_over_mass_varBeff(zcl,lob,ztr,bias_cl): 
    m_grid=10.**np.linspace(np.log10(min_mass4integral),15.5,50)
    hmf=dndM(m_grid,ztr) # [h^3/cMpc^3]
    bias_h=bias_halo(m_grid,ztr)
    yarray = m_grid*hmf*int_over_lambda_varBeff(lob,zcl,ztr,m_grid,bias_cl,bias_h) # [Mgrid,zgrid]
    integral=np.trapz(yarray,np.log(m_grid),axis=0)
    return integral # [h^3/cMpc^3]

def int_over_lambda_varBeff(lob,zcl,z,M,bias_cl,bias_h):
    ltr_grid=np.linspace(1.0e-10,lob,100)
    yarray=pltr_M(ltr_grid[:,None],M[None,:],z)*ltr_grid[:,None]**2.*(
                    2.*np.pi*int_over_phi_varBeff(lob,zcl,z,ltr_grid,bias_cl,bias_h))
    integral=np.trapz(yarray,ltr_grid,axis=0)
    return integral

def int_over_phi_varBeff(lob,zcl,z,ltr_grid,bias_cl,bias_h,damping=damping_sigmoid):
    theta_ltrz=R_lambda(ltr_grid) *(1.+z)/chi_of_z(z) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
    theta_lobzob=R_lambda(lob) *(1.+zcl)/chi_of_z(zcl) # theta = Rlambda / D_a(z) = Rlambda * (1+z) / chi(z)
#     theta_grid=np.linspace(0.,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    theta_grid=np.geomspace(1.0e-6,theta_lobzob+theta_ltrz,50) # [theta,ltr]
    dis=np.sqrt(chi_of_z(zcl)**2.+chi_of_z(z)**2.-2.*chi_of_z(zcl)*chi_of_z(z)*np.cos(theta_grid.reshape(-1))) # [theta,ltr]
    xi_NL = np.array([xir2_NL_r_z(np.log(dis[i]),zcl)[0][0]/dis[i]**2. for i in range(dis.size)])
    b0,b1,b2=[2.0,1./2.,0.75]
    theta_1Mpc= R_lambda(lob)/chi_of_z(zob)
    x0 = b0*theta_1Mpc
    k = b1*10. / x0  # steepness of the transition
    eff_bias_lobztr_theta=(bias_cl*b2+(bias_cl-bias_cl*b2)/(1.+np.exp(-k*(theta_grid-x0)))) # [theta,ltr]
    bM_bsel_xi=bias_h[None,None,:]*eff_bias_lobztr_theta[:,:,None]*np.reshape(xi_NL,theta_grid.shape)[:,:,None] # [theta_grid,ltr,Mgrid]
    if exclusion_sigma: bM_bsel_xi[np.reshape(dis,theta_grid.shape)<R_lambda(lob) *(1.+zob),:] = -1.
    bM_bsel_xi[bM_bsel_xi<-1.]=-1
    yarray=np.sin(theta_grid)[:,:,None]*(1.+bM_bsel_xi)*(area_overlap(theta_grid,theta_lobzob,theta_ltrz)[:,:,None])**2.   
    integral=np.trapz(yarray,theta_grid[:,:,None],axis=0) # [ltr,Mgrid]
    return integral

def area_overlap(theta, theta_lob, theta_l):
    """
    Fractional overlap area between two circular apertures in projection.

    Parameters
    ----------
    theta : array
        Angular separation between halo centers [rad].
    theta_lob : float
        Angular radius of the main halo.
    theta_l : array
        Angular radius of the secondary halo.

    Returns
    -------
    A_overlap : array
        Fraction of overlapping area.
    """
    theta_l_reshape = theta_l[None, :] * np.ones(theta.shape)

    # Initialize full overlap
    A_overlap = np.ones(theta.shape)

    # No overlap
    A_overlap[theta > theta_lob + theta_l_reshape] = 0.

    # Full containment
    mask = theta_l_reshape > theta_lob
    A_overlap[mask] = theta_lob**2. / theta_l_reshape[mask]**2.

    # Partial overlap
    cond = theta > np.abs(theta_lob - theta_l_reshape)

    argcos1 = (
        (theta[cond]**2. + theta_l_reshape[cond]**2. - theta_lob**2.)
        / (2. * theta[cond] * theta_l_reshape[cond])
    )
    argcos1 = np.clip(argcos1, -1., 1.)

    argcos2 = (
        (theta[cond]**2. + theta_lob**2. - theta_l_reshape[cond]**2.)
        / (2. * theta[cond] * theta_lob)
    )
    argcos2 = np.clip(argcos2, -1., 1.)

    argsqrt = (
        (-theta[cond] + theta_l_reshape[cond] + theta_lob)
        * (theta[cond] + theta_l_reshape[cond] - theta_lob)
        * (theta[cond] - theta_l_reshape[cond] + theta_lob)
        * (theta[cond] + theta_l_reshape[cond] + theta_lob)
    )
    argsqrt[argsqrt < 0.] = 0.

    A_overlap[cond] = (
        theta_l_reshape[cond]**2. * np.arccos(argcos1)
        + theta_lob**2. * np.arccos(argcos2)
        - 0.5 * np.sqrt(argsqrt)
    ) / (np.pi * theta_l_reshape[cond]**2.)

    return A_overlap


In [ ]:
# ## Set the number of lambda/z bin
num_l_bin=1
num_z_bin=1

lob=20.0
zcl=0.5
bin_z=np.array([zcl*(1.-0.05),zcl*(1.+0.05)])
bin_l=np.array([lob*(1-0.025),lob*(1+0.025)])
print(bin_l)


# For mock NFW profile
num_radial_bin=20
num_radial_edges=num_radial_bin+1
Sigma_r_grid=10.**np.linspace(-2.,np.log10(30.),num_radial_edges) # comoving Mpc/h
R_Sigma_r_grid=Sigma_r_grid[:-1]+0.5*np.diff(Sigma_r_grid)

i_Rbin_min=4


cond0=np.invert(np.all(Sigma_R[:,:]==0.,axis=1))

plt_counter=0
for iz in range(num_z_bin):
    condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
    for il in range(num_l_bin):
        condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
        cond=condl*condz*cond0
            
        sigma_R_mean_lob=np.mean(Sigma_R[cond,:],axis=0)
        
        sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R[cond],
                             np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R[cond0], dm=0.1, dz=0.05)
        

        
        y_hi = sigma_R_mean_lob[i_Rbin_min:] +2*np.std(Sigma_R[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        y_lo = sigma_R_mean_lob[i_Rbin_min:] -2*np.std(Sigma_R[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)

        
        plt.fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo, y_hi, alpha=0.2)
        
        
        sigma_R_mean_lob=np.mean(Sigma_R_PRJ[cond,:],axis=0)
        
        sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R_PRJ[cond],
                                                  np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R_PRJ[cond0], dm=0.1, dz=0.05)
        std_sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift_std(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R_PRJ[cond],
                             np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R_PRJ[cond0], dm=0.1, dz=0.05)        

        
        y_hi = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) + 2*np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        y_lo = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) - 2*np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        
        plt.fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo, y_hi, alpha=0.2)        
        
        y_hi = sigma_R_mean_M + std_sigma_R_mean_M
        y_lo = sigma_R_mean_M - std_sigma_R_mean_M
        
        sigma_R_mean_lob=np.mean(Sigma_R[cond,:]-Sigma_R_PRJ[cond,:],axis=0)
        y_hi = sigma_R_mean_lob[i_Rbin_min:] +2*np.std(Sigma_R[cond,i_Rbin_min:]-Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        y_lo = sigma_R_mean_lob[i_Rbin_min:] -2*np.std(Sigma_R[cond,i_Rbin_min:]-Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        plt.fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo, y_hi,color='k', alpha=0.2)
        
        
        plt.xscale('log')
        plt.yscale('log')
        plt.ylim(4.0e12,4.0e14)
        plt.xlim(0.07,25.)


        plt.xlabel('$R$ [cMpc/h]',fontsize=16)
        plt.ylabel('$\\Sigma(R|\\lambda^{\\rm ob}=20,z^{\\rm ob}=0.5)$',fontsize=16)
            
        
        plt.legend(fontsize=16,frameon=False,loc='best',ncol=1)

In [ ]:
# For mock NFW profile
num_radial_bin=20
num_radial_edges=num_radial_bin+1
Sigma_r_grid=10.**np.linspace(-2.,np.log10(30.),num_radial_edges) # comoving Mpc/h
R_Sigma_r_grid=Sigma_r_grid[:-1]+0.5*np.diff(Sigma_r_grid)
i_Rbin_min=4

zcl=0.5
bin_z=np.array([zcl*(1.-0.2),zcl*(1.+0.2)])
print(bin_z)

fig, ax = plt.subplots(nrows=3, ncols=1,sharex=True,sharey=False,gridspec_kw = {'height_ratios':[3,3,3]})
fig.subplots_adjust(hspace = 0.05) # no vertical spacing between subplots
fig.set_size_inches(6,12) # set size

cond0=np.invert(np.all(Sigma_R[:,:]==0.,axis=1))

##############################################/home/#########
ltrs=np.array([10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.])
##########################################################


iz=0
il=0
lob=20.0
bin_l=np.array([lob*(1-0.025),lob*(1+0.025)])
condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
condl*=(np.abs(catalog_lambda_tot-catalog_lambda-0.75)<=0.5)
cond=condl*condz*cond0

a_scale=1./(1.+np.mean(catalog_z[cond]))


sigma_R_mean_lob=np.mean(Sigma_R_PRJ[cond,:],axis=0)       
y_hi = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) + np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
y_lo = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) - np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        
ax[0].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo/1.0e13, y_hi/1.0e13, alpha=0.2,color='C0')  


lob=20.0
bin_l=np.array([lob*(1-0.05),lob*(1+0.05)])
condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
condl*=(np.abs(catalog_lambda_tot-catalog_lambda-2.5)<=0.5)
cond=condl*condz*cond0

a_scale=1./(1.+np.mean(catalog_z[cond]))


sigma_R_mean_lob=np.mean(Sigma_R_PRJ[cond,:],axis=0)       
y_hi = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) + np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
y_lo = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) - np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        
ax[0].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo/1.0e13, y_hi/1.0e13, alpha=0.2,color='C1')    


lob=20.0
bin_l=np.array([lob*(1-0.025),lob*(1+0.025)])
condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
condl*=(np.abs(catalog_lambda_tot-catalog_lambda-5.35)<=0.5)
cond=condl*condz*cond0

a_scale=1./(1.+np.mean(catalog_z[cond]))


sigma_R_mean_lob=np.mean(Sigma_R_PRJ[cond,:],axis=0)       
y_hi = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) + np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
y_lo = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) - np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        
ax[0].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo/1.0e13, y_hi/1.0e13, alpha=0.2,color='C2')   

           
ax[0].set_xscale('log')

ax[0].legend(fontsize=12,frameon=False,loc='best',ncol=1,)
ax[0].set_ylabel("$\\langle \\Sigma^{\\rm prj} (R|\\lambda^{\\rm ob},\\lambda^{\\rm tr},z^{\\rm ob}) \\rangle$  [$10^{13}M_\\odot h/$Mpc$^2$]",
                fontsize=12)
#################################################################
zcl=0.5
bin_z=np.array([zcl*(1.-0.05),zcl*(1.+0.05)])
lob=19.5
bin_l=np.array([lob*(1-0.025),lob*(1+0.025)])
condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
cond=condl*condz*cond0

print(np.mean(catalog_lambda_tot[cond]))

a_scale=1./(1.+np.mean(catalog_z[cond]))

sigma_R_mean_lob=np.mean(Sigma_R_PRJ[cond,:],axis=0)
sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R_PRJ[cond],
                                          np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R_PRJ[cond0], dm=0.1, dz=0.05)
std_sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift_std(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R_PRJ[cond],
                     np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R_PRJ[cond0], dm=0.1, dz=0.05)
y_hi = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) + np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
y_lo = np.mean(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0) - np.std(Sigma_R_PRJ[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)

ax[1].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo/1.0e13, y_hi/1.0e13, alpha=0.2,color='r')    


y_hi = sigma_R_mean_M + std_sigma_R_mean_M
y_lo = sigma_R_mean_M - std_sigma_R_mean_M

ax[1].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo[i_Rbin_min:]/1.0e13, y_hi[i_Rbin_min:]/1.0e13,alpha=0.2,color='gray') 


ax[1].legend(fontsize=12,frameon=False,loc='best',ncol=1,)
ax[1].set_ylabel("$\\langle \\Sigma^{\\rm prj} (R|\\lambda^{\\rm ob},z^{\\rm ob}) \\rangle$ [$10^{13}M_\\odot h/$Mpc$^2$]",
                fontsize='12')

################################################################

sigma_R_mean_lob=np.mean(Sigma_R[cond,:],axis=0)

sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R[cond],
                     np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R[cond0], dm=0.1, dz=0.05)


std_sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift_std(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R[cond],
                     np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R[cond0], dm=0.1, dz=0.05)


sig_x=np.std(Sigma_R[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
sig_tot=np.sqrt((sig_x/sigma_R_mean_M[i_Rbin_min:])**2. + (sigma_R_mean_lob[i_Rbin_min:]*std_sigma_R_mean_M[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:]**2.)**2.)

y_hi = sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:] + sig_tot
y_lo = sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:] - sig_tot

ax[2].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo, y_hi, alpha=0.2,color='C4')


ax[2].fill_between(R_Sigma_r_grid[i_Rbin_min:]+1000,y_lo,
                       y_hi, alpha=1.0,facecolor='None',edgecolor='k',label='Mock data')
ax[2].plot(R_Sigma_r_grid[i_Rbin_min:]+1000,sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:],
           ls='-',alpha=1.0,color='k',label='Model')


ax[2].legend(fontsize=12,frameon=False,loc='best',ncol=1,)

ax[2].set_ylabel("$\\langle \\Sigma(R)\\rangle_{\\lambda^{\\rm ob}{\\rm -sel}} / \\langle \\Sigma(R)\\rangle_{{\\rm RND-sel}}$")

ax[2].set_xlabel('$R$ [cMpc/h]')

ax[2].set_xlim((0.07,25.))

ax[2].axhline(1.,ls='--',color='gray')

ax[0].set_title("$\\lambda^{\\rm ob}=20, z^{\\rm ob}=0.5$")

In [ ]:
num_l_bin=4
bin_l=np.array([20.0001, 30., 45., 60., 500. ])

num_z_bin=3
bin_z=np.array([0.20,0.35,0.5,0.65])

# For mock NFW profile
num_radial_bin=20
num_radial_edges=num_radial_bin+1
Sigma_r_grid=10.**np.linspace(-2.,np.log10(30.),num_radial_edges) # comoving Mpc/h
R_Sigma_r_grid=Sigma_r_grid[:-1]+0.5*np.diff(Sigma_r_grid)

i_Rbin_min=4

fig, axs = plt.subplots(nrows=3, ncols=4,sharex=True,sharey=True,gridspec_kw = {'height_ratios':[3,3,3]})
ax=np.ravel(axs)
fig.subplots_adjust(hspace = 0.05,wspace=0.05) # no vertical spacing between subplots
fig.set_size_inches(16,12) # set size

cond0=np.invert(np.all(Sigma_R[:,:]==0.,axis=1))


plt_counter=0
for iz in range(num_z_bin):
    condz=(catalog_z>=bin_z[iz])*(catalog_z<bin_z[iz+1])
    for il in range(num_l_bin):
        condl=(catalog_lambda_tot>=bin_l[il])*(catalog_lambda_tot<bin_l[il+1])
        cond=condl*condz*cond0
        
            
        sigma_R_mean_lob=np.mean(Sigma_R[cond,:],axis=0)
        
        sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R[cond],
                             np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R[cond0], dm=0.1, dz=0.05)
        

        ax[plt_counter].plot(R_Sigma_r_grid[i_Rbin_min:],sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:],
                                 label="$\\Sigma(R)$ sim",c='C0',ls='--')
        
        std_sigma_R_mean_M=stacked_profile_weighted_by_mass_redshift_std(np.log(catalog_m200b[cond]), catalog_z[cond], Sigma_R[cond],
                             np.log(catalog_m200b[cond0]), catalog_z[cond0], Sigma_R[cond0], dm=0.1, dz=0.05)
        
        
        sig_x=np.std(Sigma_R[cond,i_Rbin_min:],axis=0)/np.sqrt(catalog_lambda[cond].size)
        sig_tot=np.sqrt((sig_x/sigma_R_mean_M[i_Rbin_min:])**2. + (sigma_R_mean_lob[i_Rbin_min:]*std_sigma_R_mean_M[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:]**2.)**2.)
        
        y_hi = sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:] + sig_tot
        y_lo = sigma_R_mean_lob[i_Rbin_min:]/sigma_R_mean_M[i_Rbin_min:] - sig_tot
        
        ax[plt_counter].fill_between(R_Sigma_r_grid[i_Rbin_min:],y_lo, y_hi, alpha=0.2)
        

        ax[plt_counter].set_xscale('log')
        ax[plt_counter].set_ylim(0.98,1.175)
        ax[plt_counter].set_xlim((0.07,25.))

            
        ax[plt_counter].text(0.09,1.12,
            "$%.2f < z < %.2f \\quad %.f < \\lambda < %.f $" %(bin_z[iz],bin_z[iz+1],bin_l[il],bin_l[il+1]),
                            fontsize=14)
        
        if plt_counter>=8: ax[plt_counter].set_xlabel('$R$ [cMpc/h]')
        if plt_counter==0 or plt_counter==4 or plt_counter==8:
            ax[plt_counter].set_ylabel(("$\\langle \\Sigma(R)\\rangle_{\\lambda^{\\rm ob}{\\rm -sel}} / \\langle \\Sigma(R)\\rangle_{{\\rm RND-sel}}$"))
        
        plt_counter +=1